In [1]:
import spacy
import pdfplumber
from spellchecker import SpellChecker

Function to return sepcific text from each chapter

In [2]:
# Path to your PDF file
pdf_path = "E:\SCHOOL\Phase 5\Kenya Constitution\Kenya Constitution.pdf"

# Function to extract text from specific pages
def extract_specific_pages(pdf_path, start_page, end_page):
    with pdfplumber.open(pdf_path) as pdf:
        extracted_text = ""
        for page_num in range(start_page, end_page):
            page = pdf.pages[page_num]
            extracted_text += page.extract_text() + "\n"
        return extracted_text

# Chapter 1

In [3]:
chapter_1 = extract_specific_pages(pdf_path, 12,14 )
chapter_1_trimmed = chapter_1.split("CHAPTER TWO")[0].strip()


In [4]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Preprocess the user query using spaCy
def preprocess_query(query):
    # Parse the query with spaCy
    doc = nlp(query)
    # Normalize the query: lowercase, lemmatize, and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Example usage of the preprocessing function
user_query = "What is the supremacy of the constitution?"
processed_query = preprocess_query(user_query)
print(processed_query)


supremacy constitution


In [5]:
def split_chapter(chapter_text):
    # Split at key headings and strip extra whitespace
    sections = {
        "sovereignty": [],
        "supremacy": [],
        "defence": []   
    }

    # Split by new lines to process line by line
    lines = chapter_text.splitlines()

    current_section = None

    for line in lines:
        stripped_line = line.strip()

        if stripped_line.startswith("Sovereignty of the people"):
            current_section = "sovereignty"
        elif stripped_line.startswith("Supremacy of this Constitution"):
            current_section = "supremacy"
        elif stripped_line.startswith("Defence of this Constitution"):
            current_section = 'defence'

        # Append line to the current section if it's set
        if current_section:
            sections[current_section].append(stripped_line)

    # Join each section into a single string
    for key in sections:
        sections[key] = "\n".join(sections[key])

    return sections

# Split the chapter into sections
chapter_1_sections = split_chapter(chapter_1_trimmed)

print("Example")
print("\nSupremacy Section:\n", chapter_1_sections['supremacy'])
print("\nDefence Section:\n", chapter_1_sections['defence'])

Example

Supremacy Section:
 Supremacy of this Constitution.
2. (1) This Constitution is the supreme law of the Republic and
binds all persons and all State organs at both levels of government.
(2) No person may claim or exercise State authority except as
authorised under this Constitution.
(3) The validity or legality of this Constitution is not subject to
challenge by or before any court or other State organ.
(4) Any law, including customary law, that is inconsistent with this
Constitution is void to the extent of the inconsistency, and any act or
omission in contravention of this Constitution is invalid.
(5) The general rules of international law shall form part of the
law of Kenya.
14 Constitution of Kenya, 2010
(6) Any treaty or convention ratified by Kenya shall form part of
the law of Kenya under this Constitution.

Defence Section:
 Defence of this Constitution.
3. (1) Every person has an obligation to respect, uphold and
defend this Constitution.
(2) Any attempt to establish a

### Question Answering mechanism for chapter 1

In [6]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Preprocess the user query using spaCy
def preprocess_query(query):
    # Parse the query with spaCy
    doc = nlp(query)
    # Normalize the query: lowercase, lemmatize, and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Define the sections variable with Chapter 1
sections = chapter_1_sections

# Define the QA mapping based on key phrases and corresponding sections
qa_mapping = {
    "supremacy": "supremacy of this constitution",
    "sovereignty": "sovereignty of the people",
    "defence": "defence of this constitution"
    # Add more mappings as needed
}

# Update the Q&A system to use preprocessed queries
def answer_question_nlp(query, sections, qa_mapping):
    # Preprocess the user query
    processed_query = preprocess_query(query)
    
    # Search for a key in qa_mapping that matches the preprocessed query
    for key in qa_mapping:
        if key in processed_query:
            # Return the relevant section in the text
            return sections[key]
            
    return "Sorry, I couldn't find an answer to your question."

# Example usage
user_query = "What is the supremacy of the constitution?"
answer = answer_question_nlp(user_query, sections, qa_mapping)
print(answer)

# Example 2
user_query_2 = 'Explain what the defence the people states'
answer_2 = answer_question_nlp(user_query_2, sections, qa_mapping)
answer_2


Supremacy of this Constitution.
2. (1) This Constitution is the supreme law of the Republic and
binds all persons and all State organs at both levels of government.
(2) No person may claim or exercise State authority except as
authorised under this Constitution.
(3) The validity or legality of this Constitution is not subject to
challenge by or before any court or other State organ.
(4) Any law, including customary law, that is inconsistent with this
Constitution is void to the extent of the inconsistency, and any act or
omission in contravention of this Constitution is invalid.
(5) The general rules of international law shall form part of the
law of Kenya.
14 Constitution of Kenya, 2010
(6) Any treaty or convention ratified by Kenya shall form part of
the law of Kenya under this Constitution.


'Defence of this Constitution.\n3. (1) Every person has an obligation to respect, uphold and\ndefend this Constitution.\n(2) Any attempt to establish a government otherwise than in\ncompliance with this Constitution is unlawful.'

# CHAPTER 1 NLU

In [7]:
# Define synonym mapping
synonyms_1 = {
    "supremacy" : ["supremacy", "authority", "ultimate power"],
    "sovereignty" : ["sovereignty", "power of the people", "authority of the people"],
    "defence" : ["defense", "protection", "preservation"]
}

# QA mapping
qa_mapping = {
    "supremacy": "supremacy",
    "sovereignty": "sovereignty",
    "defence": "defence"
}

sections = chapter_1_sections

# Preprocess the query
def preprocess_query(query):
    doc = nlp(query)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Function to correct the spelling
def correct_spelling(processed_query):
    spell = SpellChecker()
    words = processed_query.split()
    # Find misspelled words
    misspelled_words = spell.unknown(words)


    corrected_words = []
    for word in words:
        # Correct the word if it's misspelled
        if word in misspelled_words:
            corrected_word = spell.correction(word) # Get the most likely correction
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)

    # Reconstruct the initial sentence
    corrected_input = " ".join(corrected_words)

    return corrected_input

            

# Match with synonym support
def match_with_synonyms(query, qa_mapping, synonyms_1):
    processed_query = preprocess_query(query)
    print(f"Processed Query: {processed_query}")  # Debugging line

    # Correct spelling in the processed query
    corrected_query = correct_spelling(processed_query)
    print(f"Corrected Query: {corrected_query}")  # Debugging line


    for key, value in qa_mapping.items():
        print(f"Checking key: {key}, value: {value}")  # Debugging line
        for synonym in synonyms_1.get(key, [key]):
            print(f"Trying synonym: {synonym}")  # Debugging line
            if synonym in corrected_query:
                print(f"Match found with synonym: {synonym}")  # Debugging line
                return key  # Only return section value of the key
    
    print("No match found")  # Debugging line if no match is found
    return None

# Answer function with synonym and fuzzy matching
def answer_question_nlp(query, sections, qa_mapping, synonyms):
    section_key = match_with_synonyms(query, qa_mapping, synonyms)
    
    if section_key:
        # Retrieve the relevant section from the specified chapter
        return sections.get(section_key, "Section not found.")
    
    return "Sorry, I couldn't find an answer to your question."

# Example
user_query = "What is the ultimate power of the constitution?"
answer = answer_question_nlp(user_query, sections, qa_mapping, synonyms_1)
print(answer)

# Example 2
user_query2 = "What does the constitution say about defensey?"
answer = answer_question_nlp(user_query2, sections, qa_mapping, synonyms_1)
print(answer)


Processed Query: ultimate power constitution
Corrected Query: ultimate power constitution
Checking key: supremacy, value: supremacy
Trying synonym: supremacy
Trying synonym: authority
Trying synonym: ultimate power
Match found with synonym: ultimate power
Supremacy of this Constitution.
2. (1) This Constitution is the supreme law of the Republic and
binds all persons and all State organs at both levels of government.
(2) No person may claim or exercise State authority except as
authorised under this Constitution.
(3) The validity or legality of this Constitution is not subject to
challenge by or before any court or other State organ.
(4) Any law, including customary law, that is inconsistent with this
Constitution is void to the extent of the inconsistency, and any act or
omission in contravention of this Constitution is invalid.
(5) The general rules of international law shall form part of the
law of Kenya.
14 Constitution of Kenya, 2010
(6) Any treaty or convention ratified by Kenya 

# CHAPTER 2

In [8]:
# Extract chapter 2 content
chapter_2 = extract_specific_pages(pdf_path, 13,16)
chapter_2_cleaned = chapter_2.split("CHAPTER TWO")[1].strip()
chapter_2_trimmed = chapter_2_cleaned.split("CHAPTER THREE")[0].strip()

In [9]:
def split_chapter(chapter_text):
    # Split at key headings and strip extra whitespace
    sections = {
        "declaration": [],
        "territory": [],
        "devolution": [],
        "language": [],
        "religion": [],
        "symbol": [],
        "day": [],
        "value": [],
        "culture": []
    }

    # Split by new lines to process line by line
    lines = chapter_text.splitlines()

    current_section = None

    for line in lines:
        stripped_line = line.strip()

        # Start of new sections
        if stripped_line.startswith("Declaration of the republic"):
            current_section = "declaration"
        elif stripped_line.startswith("Territory of Kenya"):
            current_section = "territory"
        elif stripped_line.startswith("Devolution and access to services"):
            current_section = 'devolution'
        elif stripped_line.startswith("National, official and other languages"):
            current_section = "language"
        elif stripped_line.startswith("State and Religion"):
            current_section = "religion"
        elif stripped_line.startswith("National symbols and national days"):
            current_section = "symbol"
        elif stripped_line.startswith("The national days are"):
            current_section = "day"
        elif stripped_line.startswith("National values and principles"):
            current_section = "value"
        elif stripped_line.startswith("Culture"):
            current_section = "culture"

        # Append line to the current section if it's set
        if current_section:
            # Prevent adding "days" content to "symbols"
            if current_section == "symbol" and "The national days are" in stripped_line:
                current_section = "day"

            sections[current_section].append(stripped_line)

    # Join each section into a single string
    for key in sections:
        sections[key] = "\n".join(sections[key])

    return sections

# Split the chapter into sections
chapter_2_sections = split_chapter(chapter_2_trimmed)

# Print to verify the results
print("\n National Symbols Section:\n", chapter_2_sections["symbol"])
print("\n National Days Section:\n", chapter_2_sections["day"])



 National Symbols Section:
 National symbols and national days.
9. (1) The national symbols of the Republic are—
(a) the national flag;
(b) the national anthem;
(c) the coat of arms; and
(d) the public seal.
(2) The national symbols are as set out in the Second Schedule.

 National Days Section:
 (3) The national days are—
(a) Madaraka Day, to be observed on 1st June;
(b) Mashujaa Day, to be observed on 20th October; and
(c) Jamhuri Day, to be observed on 12th December.
(4) A national day shall be a public holiday.
(5) Parliament may enact legislation prescribing other public
holidays, and providing for observance of public holidays.


# Question Answering mechanism

In [10]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Preprocess the user query using spaCy
def preprocess_query(query):
    # Parse the query with spaCy
    doc = nlp(query)
    # Normalize the query: lowercase, lemmatize, and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Define the sections variable with Chapter 2
sections = chapter_2_sections

# Define the QA mapping based on key phrases and corresponding sections
qa_mapping = {
    "declaration": "declaration",
    "territory": "territory", 
    "devolution": "devolution",
    "language": "language",
    "religion": "state and religion",
    "symbol": "symbol",
    "day": "day",
    "value": "value",
    "culture": "culture"
}



# Update the Q&A system to use preprocessed queries
def answer_question_nlp(query, sections, qa_mapping):
    # Preprocess the user query
    processed_query = preprocess_query(query)

    # Debug
    print(f"Processed query: {processed_query}")
    
    # Search for a key in qa_mapping that matches the preprocessed query
    for key in qa_mapping:
        if key in processed_query:
            # Debug line
            print(f"key: {key}")
            # Return the relevant section in the text
            return sections[key]
        
    
    # Search for a key in qa_mapping that matches the preprocessed query
    for key in qa_mapping:
        if key in processed_query:
            # Debug line
            print(f"key: {key}")
            # Return the relevant section in the text
            return sections[key]
        
    return "Sorry, I couldn't find an answer to your question."

# Example usage
user_query = "National days in the constitution?"
answer = answer_question_nlp(user_query, sections, qa_mapping)
print(answer)

print('\n')

user_query2 = "What does the culture entail?"
answer = answer_question_nlp(user_query2, sections, qa_mapping)
print(answer)

Processed query: national day constitution
key: day
(3) The national days are—
(a) Madaraka Day, to be observed on 1st June;
(b) Mashujaa Day, to be observed on 20th October; and
(c) Jamhuri Day, to be observed on 12th December.
(4) A national day shall be a public holiday.
(5) Parliament may enact legislation prescribing other public
holidays, and providing for observance of public holidays.


Processed query: culture entail
key: culture
Culture.
11. (1) This Constitution recognises culture as the foundation of
the nation and as the cumulative civilization of the Kenyan people and
nation.
(2) The State shall—
(a) promote all forms of national and cultural expression through
literature, the arts, traditional celebrations, science,
communication, information, mass media, publications,
libraries and other cultural heritage;
(b) recognise the role of science and indigenous technologies in
the development of the nation; and
(c) promote the intellectual property rights of the people of Keny

# CHAPTER 2 NLU

In [11]:
# Define synonym mapping
synonyms_2 = {
    "declaration": ["declaration", "proclamation"],
    "territory": ["territory", "land", "region",],
    "devolution": ["devolution", "decentralization"],
    "language": ["language", "dialects", "official languages"],
    "religion": ["religion", "faith", "belief",],
    "symbol": ["symbol", "emblem", "insignia"],
    "day": ["day", "holiday", "public holiday"],
    "value": ["value", "principle", "ethic",],
    "culture": ["culture", "heritage", "tradition"]
    
}

# QA mapping
qa_mapping = {
    "declaration": "declaration",
    "territory": "territory", 
    "devolution": "devolution",
    "language": "language",
    "religion": "religion",
    "symbol": "symbol",
    "day": "day",
    "value": "value",
    "culture": "culture"
}

sections = chapter_2_sections

# Preprocess the query
def preprocess_query(query):
    doc = nlp(query)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Function to correct the spelling
def correct_spelling(processed_query):
    spell = SpellChecker()
    words = processed_query.split()
    # Find misspelled words
    misspelled_words = spell.unknown(words)


    corrected_words = []
    for word in words:
        # Correct the word if it's misspelled
        if word in misspelled_words:
            corrected_word = spell.correction(word) # Get the most likely correction
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)

    # Reconstruct the initial sentence
    corrected_input = " ".join(corrected_words)

    return corrected_input

            

# Match with synonym support
def match_with_synonyms(query, qa_mapping, synonyms_2):
    processed_query = preprocess_query(query)
    print(f"Processed Query: {processed_query}")  # Debugging line

    # Correct spelling in the processed query
    corrected_query = correct_spelling(processed_query)
    print(f"Corrected Query: {corrected_query}")  # Debugging line


    for key, value in qa_mapping.items():
        print(f"Checking key: {key}, value: {value}")  # Debugging line
        for synonym in synonyms_2.get(key, [key]):
            print(f"Trying synonym: {synonym}")  # Debugging line
            if synonym in corrected_query:
                print(f"Match found with synonym: {synonym}")  # Debugging line
                return key  # Only return section value of the key
    
    print("No match found")  # Debugging line if no match is found
    return None

# Answer function with synonym and fuzzy matching
def answer_question_nlp(query, sections, qa_mapping, synonyms):
    section_key = match_with_synonyms(query, qa_mapping, synonyms)
    
    if section_key:
        # Retrieve the relevant section from the specified chapter
        return sections.get(section_key, "Section not found.")
    
    return "Sorry, I couldn't find an answer to your question."

# Example
user_query = "What of heritage?"
answer = answer_question_nlp(user_query, sections, qa_mapping, synonyms_2)
print(answer)

# Example 2
user_query2 = "What does the constitution say about insignia?"
answer = answer_question_nlp(user_query2, sections, qa_mapping, synonyms_2)
print(answer)


Processed Query: heritage
Corrected Query: heritage
Checking key: declaration, value: declaration
Trying synonym: declaration
Trying synonym: proclamation
Checking key: territory, value: territory
Trying synonym: territory
Trying synonym: land
Trying synonym: region
Checking key: devolution, value: devolution
Trying synonym: devolution
Trying synonym: decentralization
Checking key: language, value: language
Trying synonym: language
Trying synonym: dialects
Trying synonym: official languages
Checking key: religion, value: religion
Trying synonym: religion
Trying synonym: faith
Trying synonym: belief
Checking key: symbol, value: symbol
Trying synonym: symbol
Trying synonym: emblem
Trying synonym: insignia
Checking key: day, value: day
Trying synonym: day
Trying synonym: holiday
Trying synonym: public holiday
Checking key: value, value: value
Trying synonym: value
Trying synonym: principle
Trying synonym: ethic
Checking key: culture, value: culture
Trying synonym: culture
Trying synonym: 

# CHAPTER 3

In [12]:
# Extract the relevant pages
chapter_3 = extract_specific_pages(pdf_path, 15,19 )
chapter_3_cleaned = chapter_3.split("CHAPTER THREE")[1].strip()
chapter_3_trimmed = chapter_3_cleaned.split("CHAPTER FOUR")[0].strip()


In [13]:
def split_chapter(chapter_text):
    # Split at key headings and strip extra whitespace
    sections = {
        "entitlement": [],
        "retention": [],
        "birth": [],
        "registration": [],
        "dual": [],
        "revocation": [],
        "legislation" : [],
    }

    # Split by new lines to process line by line
    lines = chapter_text.splitlines()

    current_section = None

    for line in lines:
        stripped_line = line.strip()

        if stripped_line.startswith("Entitlements of citizens"):
            current_section = "entitlement"
        elif stripped_line.startswith("Retention and acquisition of citizenship"):
            current_section = "retention"
        elif stripped_line.startswith("Citizenship by birth"):
            current_section = 'birth'
        elif stripped_line.startswith("Citizenship by registration"):
            current_section = "registration"
        elif stripped_line.startswith("Dual citizenship"):
            current_section = "dual"
        elif stripped_line.startswith("Revocation of citizenship"):
            current_section = "revocation"
        elif stripped_line.startswith("Legislation on citizenship"):
            current_section = "legislation"

        # Append line to the current section if it's set
        if current_section:
            sections[current_section].append(stripped_line)

    # Join each section into a single string
    for key in sections:
        sections[key] = "\n".join(sections[key])

    return sections

# Split the chapter into sections
chapter_3_sections = split_chapter(chapter_3_trimmed)

print("Example")
print("\nSupremacy Section:\n", chapter_3_sections['revocation'])
print('\n')
print("\nDefence Section:\n", chapter_3_sections['entitlement'])

Example

Supremacy Section:
 Revocation of citizenship.
17. (1) If a person acquired citizenship by registration, the
citizenship may be revoked if the person—
(a) acquired the citizenship by fraud, false representation or
concealment of any material fact;
(b) has, during any war in which Kenya was engaged, unlawfully
traded or communicated with an enemy or been engaged in
or associated with any business that was knowingly carried
on in such a manner as to assist an enemy in that war;
(c) has, within five years after registration, been convicted of an
offence and sentenced to imprisonment for a term of three
years or longer; or
(d) has, at any time after registration, been convicted of treason,
or of an offence for which—
(i) a penalty of at least seven years imprisonment may be
imposed; or
(ii) a more severe penalty may be imposed.
(2) The citizenship of a person who was presumed to be a citizen
by birth, as contemplated in Article 14 (4), may be revoked if—
(a) the citizenship was ac

#### Question Answering mechanism for chapter 3

In [14]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Preprocess the user query using spaCy
def preprocess_query(query):
    # Parse the query with spaCy
    doc = nlp(query)
    # Normalize the query: lowercase, lemmatize, and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Define the sections variable with Chapter 1
sections = chapter_3_sections

# Define the QA mapping based on key phrases and corresponding sections
qa_mapping = {
    "entitlement": "entitlement",
    "retention": "retention",  # Ensure this matches your sections key
    "birth": "birth",
    "registration": "registration",
    "dual": "dual",
    "revocation": "revocation",
    "legislation": "legislation"
    
}

# Update the Q&A system to use preprocessed queries
def answer_question_nlp(query, sections, qa_mapping):
    # Preprocess the user query
    processed_query = preprocess_query(query)
    
    # Search for a key in qa_mapping that matches the preprocessed query
    for key in qa_mapping:
        if key in processed_query:
            # Return the relevant section in the text
            return sections[key]
            
    return "Sorry, I couldn't find an answer to your question."

# Example usage
user_query = "dual citizenship?"
answer = answer_question_nlp(user_query, sections, qa_mapping)
print(answer)

# Example 2
user_query_2 = "how do I become a citizen by registration"
answer_2 = answer_question_nlp(user_query_2, sections, qa_mapping)
print(answer_2)


Dual citizenship.
16. A citizen by birth does not lose citizenship by acquiring the
citizenship of another country.
Citizenship by registration.
15. (1) A person who has been married to a citizen for a period of
at least seven years is entitled on application to be registered as a
citizen.
(2) A person who has been lawfully resident in Kenya for a
continuous period of at least seven years, and who satisfies the
conditions prescribed by an Act of Parliament, may apply to be
registered as a citizen.
(3) A child who is not a citizen, but is adopted by a citizen, is
entitled on application to be registered as a citizen.
18 Constitution of Kenya, 2010
(4) Parliament shall enact legislation establishing conditions on
which citizenship may be granted to individuals who are citizens of
other countries.
(5) This Article applies to a person as from the effective date, but
any requirements that must be satisfied before the person is entitled to
be registered as a citizen shall be regarded as havi

# CHAPTER 3 NLU

In [15]:
citizenship_mapping = {
    "dual citizenship": "dual",
    "retention of citizenship": "retention",
    "citizenship by birth": "birth",
    "citizenship by registration": "registration",
    "legislation on citizenship": "legislation",
    "revocation of citizenship": "revocation"
}


for subtopic, section_key in citizenship_mapping.items():
    print(section_key)

dual
retention
birth
registration
legislation
revocation


In [16]:
# Define synonym mapping
synonyms_3 = {
    "entitlement": ["entitlement", "eligibility", "entitlement rights"],
    "retention": ["retention", "maintenance"],
    "birth": ["birth", "native", "inborn citizenship"],
    "registration": ["registration", "citizenship application", "naturalization"],
    "dual": ["dual", "dual citizenship"],
    "revocation": ["revocation", "forfeiture"],
    "legislation": ["legislation", "legislation on citizenship"]

}

# QA mapping
qa_mapping = {
    "entitlement": "entitlement",
    "retention": "retention",
    "birth": "birth",
    "registration": "registration",
    "dual": "dual",
    "revocation": "revocation",
    "legislation" : "legislation"
}

# Define a dedicated mapping for citizenship subtopics
citizenship_mapping = {
    "retention citizenship": "retention",
    "citizenship birth": "birth",
    "citizenship registration": "registration",
    "dual citizenship": "dual",
    "revocation citizenship": "revocation",
    "legislation citizenship": "legislation"
}


sections = chapter_3_sections

# Preprocess the query
def preprocess_query(query):
    doc = nlp(query)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Function to correct the spelling
def correct_spelling(processed_query):
    spell = SpellChecker()
    words = processed_query.split()
    # Find misspelled words
    misspelled_words = spell.unknown(words)


    corrected_words = []
    for word in words:
        # Correct the word if it's misspelled
        if word in misspelled_words:
            corrected_word = spell.correction(word) # Get the most likely correction
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)

    # Reconstruct the initial sentence
    corrected_input = " ".join(corrected_words)

    return corrected_input


# Match with synonym support
def match_with_synonyms(query, qa_mapping, synonyms, citizenship_mapping):
    processed_query = preprocess_query(query)
    print(f"Processed Query: {processed_query}")  # Debugging line

    # Correct spelling in the processed query
    corrected_query = correct_spelling(processed_query)
    print(f"Corrected Query: {corrected_query}")  # Debugging line

    # Check for specific citizenship subtopics first
    for subtopic, section_key in citizenship_mapping.items():
        if subtopic in corrected_query:
            print(f"Specific citizenship topic detected: {subtopic}")
            return section_key  # Return the section key for the specific subtopic

    # Only check for general citizenship if no specific subtopic was found
    if "citizenship" in corrected_query:
        print("General citizenship query detected.")  # Debugging line
        return "citizenship"  # Return a placeholder indicating interest in citizenship

    for key in qa_mapping:
        print(f"Checking key: {key}")  # Debugging line
        for synonym in synonyms_3.get(key, [key]):
            print(f"Trying synonym: {synonym}")  # Debugging line
            if synonym in corrected_query:
                print(f"Match found with synonym: {synonym}")  # Debugging line
                return key  # Only return section value of the key

    print("No match found")  # Debugging line if no match is found
    return None

# Answer function with synonym and fuzzy matching
def answer_question_nlp(query, sections, qa_mapping, synonyms):
    section_key = match_with_synonyms(query, qa_mapping, synonyms, citizenship_mapping)
    
    if section_key in sections:
        # Retrieve the relevant section from the specified chapter
        return sections.get(section_key, "Section not found.")
    elif section_key == "citizenship":
        return (f"It seems you're interested in citizenship. "
                f"Available subtopics include: {list(citizenship_mapping.keys())}.")
    
    return "Sorry, I couldn't find an answer to your question."

# Example
user_query = "What about revocation of citizenship?"
answer = answer_question_nlp(user_query, sections, qa_mapping, synonyms_3)
print(answer)

Processed Query: revocation citizenship
Corrected Query: revocation citizenship
Specific citizenship topic detected: revocation citizenship
Revocation of citizenship.
17. (1) If a person acquired citizenship by registration, the
citizenship may be revoked if the person—
(a) acquired the citizenship by fraud, false representation or
concealment of any material fact;
(b) has, during any war in which Kenya was engaged, unlawfully
traded or communicated with an enemy or been engaged in
or associated with any business that was knowingly carried
on in such a manner as to assist an enemy in that war;
(c) has, within five years after registration, been convicted of an
offence and sentenced to imprisonment for a term of three
years or longer; or
(d) has, at any time after registration, been convicted of treason,
or of an offence for which—
(i) a penalty of at least seven years imprisonment may be
imposed; or
(ii) a more severe penalty may be imposed.
(2) The citizenship of a person who was pres

# Since all functions above run perfectly, we'll combine the constants into a single variable for easy reference in the final code,

In [17]:
# Define the synonyms explicitly so that we can easily reference them later
synonyms = {
    "supremacy": ["supremacy", "authority", "ultimate power"],
    "sovereignty": ["sovereignty", "power of the people", "authority of the people", "self rule", "autonomy"],
    "defence": ["defense", "protection", "preservation"],
    "declaration": ["declaration", "proclamation", "statement", "announcement", "affirmation"],
    "territory": ["territory", "land", "region", "area", "jurisdiction", "bounds"],
    "devolution": ["devolution", "decentralization", "delegation", "transfer of power", "local governance", "subsidiarity"],
    "languages": ["languages", "tongues", "dialects", "official languages", "linguistic diversity"],
    "religion": ["religion", "faith", "belief systems", "spiritual practice", "secularism", "church-state separation"],
    "symbol": ["symbol", "emblem", "insignia", "representation", "national icon"],
    "day": ["day", "holiday", "observance", "public holiday", "commemoration", "remembrance"],
    "value": ["value", "principle", "ethic", "core value", "standard", "national ideal"],
    "governance": ["governance", "government", "administration", "management", "public service", "political structure"],
    "culture": ["culture", "heritage", "tradition", "customs", "societal norms", "arts"],
    "entitlement": ["entitlement", "right", "eligibility", "entitlement rights", "benefits", "privileges"],
    "retention": ["retention", "maintenance", "keeping", "preservation", "continuation"],
    "birth": ["birth", "nativity", "origin", "ancestry", "inborn citizenship"],
    "registration": ["registration", "enlistment", "enrollment", "citizenship application", "naturalization"],
    "dual": ["dual", "multiple", "dual nationality", "two-fold citizenship"],
    "revocation": ["revocation", "cancellation", "annulment", "rescission", "forfeiture", "withdrawal"],
    "legislation": ["legislation", "laws", "legal framework", "statutes", "enactment"]
}

# Define qa mapping explicitly too
qa_mapping = {
    "supremacy": "supremacy",
    "sovereignty": "sovereignty",
    "defence": "defence",
    "declaration": "declaration",
    "territory": "territory", 
    "devolution": "devolution",
    "language": "languages",
    "religion": "religion",
    "symbol": "symbol",
    "day": "day",
    "value": "value",
    "culture": "culture",
    "entitlement": "entitlement",
    "retention": "retention",  # Ensure this matches your sections key
    "birth": "birth",
    "registration": "registration",
    "dual": "dual",
    "revocation": "revocation",
    "legislation": "legislation"
    
}

combined_sections = {**chapter_1_sections, **chapter_2_sections, **chapter_3_sections}
# Print the keys of the dictionary
combined_sections.keys()

dict_keys(['sovereignty', 'supremacy', 'defence', 'declaration', 'territory', 'devolution', 'language', 'religion', 'symbol', 'day', 'value', 'culture', 'entitlement', 'retention', 'birth', 'registration', 'dual', 'revocation', 'legislation'])

#

# CHAPTER 4

In [18]:
# extracting chapter 4 and trimming only to contain chapter 4 text
chapter_4 = extract_specific_pages(pdf_path, 18,41 )
chapter_4_trimmed = chapter_4.split("CHAPTER FOUR", 1)[1].split("CHAPTER FIVE", 1)[0].strip()

In [19]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Preprocess the user query using spaCy
def preprocess_query(query):
    # Parse the query with spaCy
    doc = nlp(query)
    # Normalize the query: lowercase, lemmatize, and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Example usage of the preprocessing function
user_query = "What is the supremacy of the constitution?"
processed_query = preprocess_query(user_query)
print(processed_query)

supremacy constitution


In [20]:
def split_chapter_4(chapter_text):
    # Split at key headings and strip extra whitespace
    sections = {
        "fundamental": [],
        "application": [],
        "implementation": [],
        "enforcement": [],
        "authority": [],
        "limitation": [],
        "limit": [],
        "life": [],
        "equality": [],
        "dignity": [],
        "security": [],
        "slavery": [],
        "privacy": [],
        "conscience": [],
        "expression": [],
        "medium": [],
        "information": [],
        "association": [],
        "assembly": [],
        "political": [],
        "movement": [],
        "property": [],
        "work": [],
        "environment": [],
        "economic": [],
        "language": [],
        "family": [],
        "consumer": [],
        "administrative": [],
        "justice": [],
        "arrest": [],
        "fair hearing": [],
        "custody": [],
        "interpret": [],
        "infant": [],
        "disable": [],
        "youth": [],
        "minority": [],
        "old": [],
        "emergency": [],
        "national": []
    }

    # Split by new lines to process line by line
    lines = chapter_text.splitlines()

    current_section = None

    for line in lines:
        stripped_line = line.strip()

        if stripped_line.startswith("Rights and fundamental freedoms"):
            current_section = "fundamental"
        elif stripped_line.startswith("Application of Bill of Rights"):
            current_section = "application"
        elif stripped_line.startswith("Implementation of rights and fundamental freedoms"):
            current_section = "implementation"
        elif stripped_line.startswith("Enforcement of Bill of Rights"):
            current_section = "enforcement"
        elif stripped_line.startswith("Authority of courts to uphold and enforce the Bill of Rights"):
            current_section = "authority"
        elif stripped_line.startswith("Limitation of rights and fundamental freedoms"):
            current_section = "limitation"
        elif stripped_line.startswith("Fundamental Rights and freedoms that may not be limited"):
            current_section = "limit"
        elif stripped_line.startswith("Right to life"):
            current_section = "life"
        elif stripped_line.startswith("Equality and freedom from discrimination"):
            current_section = "equality"
        elif stripped_line.startswith("Human dignity"):
            current_section = "dignity"
        elif stripped_line.startswith("Freedom and security of the person"):
            current_section = "security"
        elif stripped_line.startswith("Slavery, servitude and forced labour"):
            current_section = "slavery"
        elif stripped_line.startswith("Privacy"):
            current_section = "privacy"
        elif stripped_line.startswith("Freedom of conscience, religion, belief and opinion"):
            current_section = "conscience"
        elif stripped_line.startswith("Freedom of expression"):
            current_section = "expression"
        elif stripped_line.startswith("Freedom of the media"):
            current_section = "medium"
        elif stripped_line.startswith("Access to information"):
            current_section = "information"
        elif stripped_line.startswith("Freedom of association"):
            current_section = "association"
        elif stripped_line.startswith("Assembly, demonstration, picketing and petition"):
            current_section = "assembly"
        elif stripped_line.startswith("Political rights"):
            current_section = "political"
        elif stripped_line.startswith("Freedom of movement and residence"):
            current_section = "movement"
        elif stripped_line.startswith("Protection of right to property"):
            current_section = "property"
        elif stripped_line.startswith("Labour relations"):
            current_section = "work"
        elif stripped_line.startswith("Environment"):
            current_section = "environment"
        elif stripped_line.startswith("Economic and social rights"):
            current_section = "economic"
        elif stripped_line.startswith("Language and culture"):
            current_section = "language"
        elif stripped_line.startswith("Family"):
            current_section = "family"
        elif stripped_line.startswith("Consumer rights"):
            current_section = "consumer"
        elif stripped_line.startswith("Fair administrative action"):
            current_section = "administrative"
        elif stripped_line.startswith("Access to justice"):
            current_section = "justice"
        elif stripped_line.startswith("Rights of arrested persons"):
            current_section = "arrest"
        elif stripped_line.startswith("Fair hearing"):
            current_section = "fair hearing"
        elif stripped_line.startswith("Rights of persons detained, held in custody or imprisoned"):
            current_section = "custody"
        elif stripped_line.startswith("Interpretation of this Part"):
            current_section = "interpret"
        elif stripped_line.startswith("Children"):
            current_section = "infant"
        elif stripped_line.startswith("Persons with disabilities"):
            current_section = "disable"
        elif stripped_line.startswith("Youth"):
            current_section = "youth"
        elif stripped_line.startswith("Minorities and marginalised groups"):
            current_section = "minority"
        elif stripped_line.startswith("Older members of society"):
            current_section = "old"
        elif stripped_line.startswith("State of emergency"):
            current_section = "emergency"
        elif stripped_line.startswith("Kenya National Human Rights and Equality Commission"):
            current_section = "national"
        # Append line to the current section if it's set
        if current_section:
            sections[current_section].append(stripped_line)

    # Join each section into a single string
    for key in sections:
        sections[key] = "\n".join(sections[key])

    return sections

# Split the chapter into sections
chapter_4_sections = split_chapter_4(chapter_4_trimmed)

print("Fundamental Rights and freedoms that may not be limited Section:\n", chapter_4_sections['limit'])

Fundamental Rights and freedoms that may not be limited Section:
 Fundamental Rights and freedoms that may not be limited.
25. Despite any other provision in this Constitution, the following
rights and fundamental freedoms shall not be limited—
(a) freedom from torture and cruel, inhuman or degrading
treatment or punishment;
24 Constitution of Kenya, 2010
(b) freedom from slavery or servitude;
(c) the right to a fair trial; and
(d) the right to an order of habeas corpus.
PART 2—RIGHTS AND FUNDAMENTAL FREEDOMS


### Question Answering mechanism for chapter 4

In [21]:
# Define the sections variable with Chapter 1
sections = chapter_4_sections

# Define the QA mapping based on key phrases and corresponding sections
qa_mapping = {
    "fundamental": "rights and fundamental freedoms",
    "application": "application of Bill of Rights", 
    "implementation": "implementation of rights and fundamental freedoms", 
    "enforcement": "enforcement of Bill of Rights", 
    "authority": "authority of courts to uphold and enforce the Bill of Rights", 
    "limitation": "limitation of rights and fundamental freedoms", 
    "limit": "fundamental Rights and freedoms that may not be limited", 
    "life": "right to life", 
    "equality": "equality and freedom from discrimination", 
    "dignity": "human dignity", 
    "security": "freedom and security of the person", 
    "slavery": "slavery, servitude and forced labour", 
    "privacy": "privacy", 
    "conscience": "freedom of conscience, religion, belief and opinion", 
    "expression": "freedom of expression", 
    "medium": "freedom of the media", 
    "information": "access to information", 
    "association": "freedom of association", 
    "assembly": "assembly, demonstration, picketing and petition", 
    "political": "political rights", 
    "movement": "freedom of movement and residence", 
    "property": "protection of right to property", 
    "work": "labour relations", 
    "environment": "environment", 
    "economic": "economic and social rights", 
    "language": "language and culture", 
    "family": "family", 
    "consumer": "consumer rights", 
    "administrative": "fair administrative action", 
    "justice": "access to justice" ,
    "arrest": "rights of arrested persons", 
    "fair hearing": "fair hearing", 
    "custody": "rights of persons detained, held in custody or imprisoned", 
    "interpret": "interpretation of this Part", 
    "infant": "children", 
    "disable": "persons with disabilities", 
    "youth": "youth", 
    "minority": "minorities and marginalised groups", 
    "old": "older members of society", 
    "emergency": "state of emergency",
    "national": "kenya national human rights and equality Commission"
}

# Update the Q&A system to use preprocessed queries
def answer_question_nlp(query, sections, qa_mapping):
    # Preprocess the user query
    processed_query = preprocess_query(query)
    
    # Search for a key in qa_mapping that matches the preprocessed query
    for key in qa_mapping:
        if key in processed_query:
            # Return the relevant section in the text
            return sections[key]
            
    return "Sorry, I couldn't find an answer to your question."

# Example usage
user_query = "What is infant?"
answer = answer_question_nlp(user_query, sections, qa_mapping)
print(answer)

Children.
53. (1) Every child has the right—
(a) to a name and nationality from birth;
(b) to free and compulsory basic education;
(c) to basic nutrition, shelter and health care;
Constitution of Kenya, 2010 37
(d) to be protected from abuse, neglect, harmful cultural
practices, all forms of violence, inhuman treatment and
punishment, and hazardous or exploitative labour;
(e) to parental care and protection, which includes equal
responsibility of the mother and father to provide for the child,
whether they are married to each other or not; and
(f) not to be detained, except as a measure of last resort, and
when detained, to be held—
(i) for the shortest appropriate period of time; and
(ii) separate from adults and in conditions that take account
of the child’s sex and age.
(2) A child’s best interests are of paramount importance in every
matter concerning the child.


# CHAPTER 4 NLU

In [22]:
# Define synonym mapping
synonyms = {
    "fundamental": ["fundamental", "fundamental freedoms", "essential"],
    "application": ["application", "exercise", "freedom enforcement"],
    "implementation": ["implementation", "right support", "execution"],
    "authority": ["authority", "judicial power", "court jurisdiction", "judicial review"],
    "limitation": ["limitation", "freedom restriction", "limit entitlement"],
    "limit": ["limit","absolute right", "inalienable freedom", "immutable right",],
    "life": ["life", "existence", "entitlement to life", "survive"],
    "equality": ["equality", "equal treatment", "justice"],
    "dignity": ["dignity", "intrinsic respect", "inherent value", "personal honor"],
    "security": ["security", "liberty", "personal safety"],
    "slavery": ["slavery", "bondage", "force servitude", "involuntary labor", "coerce work"],
    "privacy": ["privacy", "confidential", "personal space"],
    "conscience": ["conscience", "liberty of thought", "religion freedom", "belief"],
    "expression": ["expression", "communication", "speak"],
    "medium": ["medium", "press autonomy", "journalist freedom", "right to information"],
    "information": ["information", "transparent", "datum"],
    "association": ["association", "union", "group formation", "association right"],
    "assembly": ["assembly", "dissent", "peace assembly", "demonstrate", "petition right"],
    "political": ["political", "electoral right", "political engagement", "vote right"],
    "movement": ["movement", "travel", "resident freedom", "movement liberty"],
    "Property": ["Property", "property safeguard", "assets security"],
    "work": ["work", "benefit", "previlage"],
    "environment": ["environment", "conserve", "right to a healthy environment"],
    "economic": ["economic", "standard of life", "social welfare"],
    "language": ["language", "cultural right", "linguistic freedom", "cultural expression"],
    "family": ["family", "society foundation", "parent right"],
    "consumer": ["consumer", "client entitlement", "buyer safeguard"],
    "administrative": ["administrative", "fair administration rule", "legal administration measure"],
    "justice": ["justice", "legal access", "justice available"],
    "arrest": ["arrest", "capture", "seizure", "apprehension"],
    "fair hearing": ["fair hearing", "prosecute", "equity hearing"],
    "custody": ["custody", "confinement", "imprisonment", "prison"],
    "interpret": ["interpret", "explain", "clarify", "overview"],
    "infant": ["infant", "child", "toddler", "kid"],
    "disable": ["disable", "handicap", "impairment", "challenge"],
    "youth": ["youth", "young adult", "adolescent"],
    "minority": ["minority", "diversity right", "affirmative right"],
    "old": ["old", "elder", "vetran", "vintage"],
    "emergency": ["emergency", "danger", "crisis", "disaster"],
    "national": ["national", "human right", "equaity Body", "right authority"]
}

# QA mapping
qa_mapping = {
    "fundamental": ("fundamental"),
    "application": ("application"),
    "implementation": ("implementation"),
    "authority": ("authority"),
    "limitation": ("limitation"),
    "limit": ("limit"),
    "life": ("life"),
    "equality": ("equality"),
    "dignity": ("dignity"),
    "security": ("security"),
    "slavery": ("slavery"),
    "privacy": ("privacy"),
    "conscience": ("conscience"),
    "expression": ("expression"),
    "medium": ("medium"),
    "information": ("information"),
    "association": ("association"),
    "assembly": ("assembly"),
    "political": ("political"),
    "movement": ("movement"),
    "property": ("property"),
    "work": ("work"),
    "environment": ("environment"),
    "economic": ("economic"),
    "language": ("language"),
    "family": ("family"),
    "consumer": ("consumer"),
    "administrative": ("administrative"),
    "justice": ("justice"),
    "arrest": ("arrest"),
    "fair hearing": ("fair hearing"),
    "custody": ("custody"),
    "interpret": ("interpret"),
    "infant": ("infant"),
    "disable": ("disable"),
    "youth": ("youth"),
    "minority": ("minority"),
    "old": ("old"),
    "emergency": ("emergency"),
    "national": ("national")
}

# Preprocess the query
def preprocess_query(query):
    doc = nlp(query)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Function to correct the spelling
def correct_spelling(processed_query):
    spell = SpellChecker()
    words = processed_query.split()
    # Find misspelled words
    misspelled_words = spell.unknown(words)


    corrected_words = []
    for word in words:
        # Correct the word if it's misspelled
        if word in misspelled_words:
            corrected_word = spell.correction(word) # Get the most likely correction
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)

    # Reconstruct the initial sentence
    corrected_input = " ".join(corrected_words)

    return corrected_input

            

# Match with synonym support
def match_with_synonyms(query, qa_mapping, synonyms):
    processed_query = preprocess_query(query)
    print(f"Processed Query: {processed_query}")  # Debugging line

    # Correct spelling in the processed query
    corrected_query = correct_spelling(processed_query)
    print(f"Corrected Query: {corrected_query}")  # Debugging line


    for key, value in qa_mapping.items():
        print(f"Checking key: {key}, value: {value}")  # Debugging line
        for synonym in synonyms.get(key, [key]):
            print(f"Trying synonym: {synonym}")  # Debugging line
            if synonym in corrected_query:
                print(f"Match found with synonym: {synonym}")  # Debugging line
                return key  # Only return section value of the key
    
    print("No match found")  # Debugging line if no match is found
    return None

# Answer function with synonym and fuzzy matching
def answer_question_nlp(query, sections, qa_mapping, synonyms):
    section_key = match_with_synonyms(query, qa_mapping, synonyms)
    
    if section_key:
        # Retrieve the relevant section from the specified chapter
        return sections.get(section_key, "Section not found.")
    
    return "Sorry, I couldn't find an answer to your question."

# Example
user_query = "What is infant?"
answer = answer_question_nlp(user_query, sections, qa_mapping, synonyms)
print(answer)

Processed Query: infant
Corrected Query: infant
Checking key: fundamental, value: fundamental
Trying synonym: fundamental
Trying synonym: fundamental freedoms
Trying synonym: essential
Checking key: application, value: application
Trying synonym: application
Trying synonym: exercise
Trying synonym: freedom enforcement
Checking key: implementation, value: implementation
Trying synonym: implementation
Trying synonym: right support
Trying synonym: execution
Checking key: authority, value: authority
Trying synonym: authority
Trying synonym: judicial power
Trying synonym: court jurisdiction
Trying synonym: judicial review
Checking key: limitation, value: limitation
Trying synonym: limitation
Trying synonym: freedom restriction
Trying synonym: limit entitlement
Checking key: limit, value: limit
Trying synonym: limit
Trying synonym: absolute right
Trying synonym: inalienable freedom
Trying synonym: immutable right
Checking key: life, value: life
Trying synonym: life
Trying synonym: existence


# Chapter 5

In [23]:
# extracting the whole of chapter 4 which is located in page 41 to 48 and spliting chapter 5 only
chapter_5 = extract_specific_pages(pdf_path, 40, 48)
chapter_5_trimmed = chapter_5.split("CHAPTER FIVE", 1)[1].split("CHAPTER SIX", 1)[0].strip()

In [24]:
def split_chapter_5(chapter_text):
    # Split at key headings and strip extra whitespace
    sections = {
        "principle": [],
        "classification": [],
        "public": [],
        "community": [],
        "private": [],
        "landhold": [],
        "regulation": [],
        "commission": [],
        "land": [],
        "obligation": [],
        "environmental": [],
        "resource": [],
        "legislation": []
        
    }

    # Split by new lines to process line by line
    lines = chapter_text.splitlines()

    current_section = None

    for line in lines:
        stripped_line = line.strip()

        if stripped_line.startswith("Principles of land policy"):
            current_section = "principle"
        elif stripped_line.startswith("Classification of land"):
            current_section = "classification"
        elif stripped_line.startswith("Public land"):
            current_section = "public"
        elif stripped_line.startswith("Community land"):
            current_section = "community"
        elif stripped_line.startswith("Private land"):
            current_section = "private"
        elif stripped_line.startswith("Landholding by non-citizens"):
            current_section = "landhold"
        elif stripped_line.startswith("Regulation of land use and property"):
            current_section = "regulation"
        elif stripped_line.startswith("National Land Commission"):
            current_section = "commission"
        elif stripped_line.startswith("Legislation on land"):
            current_section = "land"
        elif stripped_line.startswith("Obligations in respect of the environment"):
            current_section = "obligation"
        elif stripped_line.startswith("Enforcement of environmental rights"):
            current_section = "environmental"
        elif stripped_line.startswith("Agreements relating to natural resource"):
            current_section = "resource"
        elif stripped_line.startswith("Legislation relating to the environment"):
            current_section = "legislation"
        # Append line to the current section if it's set
        if current_section:
            sections[current_section].append(stripped_line)

    # Join each section into a single string
    for key in sections:
        sections[key] = "\n".join(sections[key])

    return sections

# Split the chapter into sections
chapter_5_sections = split_chapter_5(chapter_5_trimmed)

print("Legislation relating to the environment Section:\n", chapter_5_sections['legislation'])

Legislation relating to the environment Section:
 Legislation relating to the environment.
72. Parliament shall enact legislation to give full effect to the
provisions of this Part.


# Question Answering mechanism for chapter 5

In [25]:
sections = chapter_5_sections
##Define Q&A mapping for Chapter 5
qa_mapping = {
    "principle": "principles of land policy",
    "classification": "classification of land",
    "public": "public land",
    "community": "community land",
    "private": "private land",
    "landhold": "landholding by non-citizens" ,
    "regulation": "regulation of land use and property",
    "commission": "national Land Commission",
    "legislation": "legislation on land",
    "obligation": "obligations in respect of the environment",
    "environmental": "enforcement of environmental rights",
    "resource": "agreements relating to natural resources",
    "legislation": "legislation relating to the environment"
}

# Update the Q&A system to use preprocessed queries
def answer_question_nlp(query, sections, qa_mapping):
    # Preprocess the user query
    processed_query = preprocess_query(query)

    # Debug
    print(f"Processed query: {processed_query}")
    
    # Search for a key in qa_mapping that matches the preprocessed query
    for key in qa_mapping:
        if key in processed_query:
            # Debug line
            print(f"key: {key}")
            # Return the relevant section in the text
            return sections[key]
        
    
    # Search for a key in qa_mapping that matches the preprocessed query
    for key in qa_mapping:
        if key in processed_query:
            # Debug line
            print(f"key: {key}")
            # Return the relevant section in the text
            return sections[key]
        
    return "Sorry, I couldn't find an answer to your question."

# Example usage
user_query = "explain legislation?"
answer = answer_question_nlp(user_query, sections, qa_mapping)
print(answer)

Processed query: explain legislation
key: legislation
Legislation relating to the environment.
72. Parliament shall enact legislation to give full effect to the
provisions of this Part.


# CHAPTER 5 NLU

In [26]:
# Define synonym mapping
synonyms = {
    "principle": ["principle", "land management principle", "land policy guide"],
    "classification": ["classification", "land category", "types of land"],
    "public": ["public", "government land", "state property", "national land", "public landhold"],
    "community": ["community", "ethnic land", "cultural landhold", "community land"],
    "private": ["private", "individual landhold", "personal property", "freehold land", "private land"],
    "landhold" : ["landhold", "foreign lease", "alien land", "non-citizen land"],
    "regulation" : ["regulation", "land use policy", "property regulation", "land oversight"],
    "commission" : ["commission", "land authority", "public land commission", "land policy agency"],
    "land" : ["land", "land legislation", "property law", "land-use regulation", "land act"],
    "obligation" : ["obligation", "biodiversity duty", "conservation mandate"],
    "environmental" : ["environmental", "ecological redress", "sustainability enforcement"],
    "resource" : ["resource", "resource use agreement", "environmental concession"],
    "legislation" : ["legislation", "environmental law", "green policy enactment"]
}

# QA mapping
qa_mapping = {
    "principle": ("principle"),
    "classification": ("classification"),
    "public": ("public"),
    "community": ("community"),
    "private": ("private"),
    "landhold": ("landhold"),
    "regulation": ("regulation"),
    "commission": ("commission"),
    "land": ("land"),
    "obligation": ("Obligation"),
    "environmental": ("environmental"),
    "resource": ("resource"),
    "legislation": ("legislation")
}

sections = chapter_5_sections

# Preprocess the query
def preprocess_query(query):
    doc = nlp(query)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Function to correct the spelling
def correct_spelling(processed_query):
    spell = SpellChecker()
    words = processed_query.split()
    # Find misspelled words
    misspelled_words = spell.unknown(words)


    corrected_words = []
    for word in words:
        # Correct the word if it's misspelled
        if word in misspelled_words:
            corrected_word = spell.correction(word) # Get the most likely correction
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)

    # Reconstruct the initial sentence
    corrected_input = " ".join(corrected_words)

    return corrected_input

            

# Match with synonym support
def match_with_synonyms(query, qa_mapping, synonyms):
    processed_query = preprocess_query(query)
    print(f"Processed Query: {processed_query}")  # Debugging line

    # Correct spelling in the processed query
    corrected_query = correct_spelling(processed_query)
    print(f"Corrected Query: {corrected_query}")  # Debugging line


    for key, value in qa_mapping.items():
        print(f"Checking key: {key}, value: {value}")  # Debugging line
        for synonym in synonyms.get(key, [key]):
            print(f"Trying synonym: {synonym}")  # Debugging line
            if synonym in corrected_query:
                print(f"Match found with synonym: {synonym}")  # Debugging line
                return key  # Only return section value of the key
    
    print("No match found")  # Debugging line if no match is found
    return None

# Answer function with synonym and fuzzy matching
def answer_question_nlp(query, sections, qa_mapping, synonyms):
    section_key = match_with_synonyms(query, qa_mapping, synonyms)
    
    if section_key:
        # Retrieve the relevant section from the specified chapter
        return sections.get(section_key, "Section not found.")
    
    return "Sorry, I couldn't find an answer to your question."

# Example
user_query = "What is principle?"
answer = answer_question_nlp(user_query, sections, qa_mapping, synonyms)
print(answer)

Processed Query: principle
Corrected Query: principle
Checking key: principle, value: principle
Trying synonym: principle
Match found with synonym: principle
Principles of land policy.
60. (1) Land in Kenya shall be held, used and managed in a
manner that is equitable, efficient, productive and sustainable, and in
accordance with the following principles—
(a) equitable access to land;
(b) security of land rights;
(c) sustainable and productive management of land resources;
(d) transparent and cost effective administration of land;
(e) sound conservation and protection of ecologically sensitive
areas;
42 Constitution of Kenya, 2010
(f) elimination of gender discrimination in law, customs and
practices related to land and property in land; and
(g) encouragement of communities to settle land disputes
through recognised local community initiatives consistent with
this Constitution.
(2) These principles shall be implemented through a national land
policy developed and reviewed regularly by 

# Chapter 6

In [27]:
# extracting the whole of chapter 6 which is located in page 48 to 51 trimming only chapter  6
chapter_6 = extract_specific_pages(pdf_path, 47, 51)
chapter_6_trimmed = chapter_6.split("CHAPTER SIX", 1)[1].split("CHAPTER SEVEN", 1)[0].strip()

In [28]:
def split_chapter_6(chapter_text):
    # Split at key headings and strip extra whitespace
    sections = {
        "responsibility": [],
        "oath": [],
        "conduct": [],
        "finance": [],
        "restrict": [],
        "citizen": [],
        "establish": [],
        "leader": []
    }

    # Split by new lines to process line by line
    lines = chapter_text.splitlines()

    current_section = None

    for line in lines:
        stripped_line = line.strip()

        if stripped_line.startswith("Responsibilities of leadership"):
            current_section = "responsibility"
        elif stripped_line.startswith("Oath of office of State officers"):
            current_section = "oath"
        elif stripped_line.startswith("Conduct of State officers"):
            current_section = "conduct"
        elif stripped_line.startswith("Financial probity of State officers"):
            current_section = "finance"
        elif stripped_line.startswith("Restriction on activities of State officers"):
            current_section = "restrict"
        elif stripped_line.startswith("Citizenship and leadership"):
            current_section = "citizen"
        elif stripped_line.startswith("Legislation to establish the ethics and anti-corruption commission"):
            current_section = "establish"
        elif stripped_line.startswith("Legislation on leadership"):
            current_section = "leader"
        # Append line to the current section if it's set
        if current_section:
            sections[current_section].append(stripped_line)

    # Join each section into a single string
    for key in sections:
        sections[key] = "\n".join(sections[key])

    return sections

# Split the chapter into sections
chapter_6_sections = split_chapter_6(chapter_6_trimmed)

print("Financial probity Section:\n", chapter_6_sections['finance'])

Financial probity Section:
 Financial probity of State officers.
76. (1) A gift or donation to a State officer on a public or official
occasion is a gift or donation to the Republic and shall be delivered to
the State unless exempted under an Act of Parliament.
(2) A State officer shall not—
(a) maintain a bank account outside Kenya except in accordance
with an Act of Parliament; or
(b) seek or accept a personal loan or benefit in circumstances
that compromise the integrity of the State officer.


### Question Answering mechanism for chapter 6

In [29]:
sections = chapter_6_sections
##Define Q&A mapping for Chapter 6
qa_mapping = {
        "responsibility": "responsibilities of leadership",
        "oath": "oath of office of State officers",
        "conduct": "conduct of State officers",
        "finance": "financial probity of State officers",
        "restrict": "restriction on activities of State officers",
        "citizen": "citizenship and leadership",
        "establish": "legislation to establish the ethics and anti-corruption commission",
        "leader": "legislation on leadership"
}

# Update the Q&A system to use preprocessed queries
def answer_question_nlp(query, sections, qa_mapping):
    # Preprocess the user query
    processed_query = preprocess_query(query)
    
    # Search for a key in qa_mapping that matches the preprocessed query
    for key in qa_mapping:
        if key in processed_query:
            # Return the relevant section in the text
            return sections[key]
            
    return "Sorry, I couldn't find an answer to your question."

# Example usage
user_query = "what does costitution state about oath?"
answer = answer_question_nlp(user_query, sections, qa_mapping)
print(answer)

Oath of office of State officers.
74. Before assuming a State office, acting in a State office, or
performing any functions of a State office, a person shall take and
subscribe the oath or affirmation of office, in the manner and form
prescribed by the Third Schedule or under an Act of Parliament.


# chapter 6 NLU

In [30]:
# Define synonym mapping
synonyms = {
    "responsibility": ["responsibility", "accountable", "commitment", "charge"],
    "oath": ["oath", "swear", "state office affirmation", "vow", "declare"],
    "conduct": ["conduct", "behavior", "comportment", "demeanor"],
    "finance": ["finance", "finance integrity", "wealth", "networth", "finance management"],
    "restrict": ["restrict", "constrain", "regulate", "prohibit", "curtail", "control"],
    "citizen": ["citizen", "inhabitant", "local resident"],
    "establish": ["establish", "corrupt", "bribe", "fraud"],
    "leader": ["leader", "govern authority", "Legislation govern leadership"]
}

# QA mapping
qa_mapping = {
    "responsibility": ("responsibility"),
    "oath": ("oath"),
    "conduct": ("conduct"),
    "finance": ("finance"),
    "restrict": ("restrict"),
    "citizen": ("citizen"),
    "establish": ("establish"),
    "leader": ("leader")
}

sections = chapter_6_sections

# Preprocess the query
def preprocess_query(query):
    doc = nlp(query)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Function to correct the spelling
def correct_spelling(processed_query):
    spell = SpellChecker()
    words = processed_query.split()
    # Find misspelled words
    misspelled_words = spell.unknown(words)


    corrected_words = []
    for word in words:
        # Correct the word if it's misspelled
        if word in misspelled_words:
            corrected_word = spell.correction(word) # Get the most likely correction
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)

    # Reconstruct the initial sentence
    corrected_input = " ".join(corrected_words)

    return corrected_input

            

# Match with synonym support
def match_with_synonyms(query, qa_mapping, synonyms):
    processed_query = preprocess_query(query)
    print(f"Processed Query: {processed_query}")  # Debugging line

    # Correct spelling in the processed query
    corrected_query = correct_spelling(processed_query)
    print(f"Corrected Query: {corrected_query}")  # Debugging line


    for key, value in qa_mapping.items():
        print(f"Checking key: {key}, value: {value}")  # Debugging line
        for synonym in synonyms.get(key, [key]):
            print(f"Trying synonym: {synonym}")  # Debugging line
            if synonym in corrected_query:
                print(f"Match found with synonym: {synonym}")  # Debugging line
                return key  # Only return section value of the key
    
    print("No match found")  # Debugging line if no match is found
    return None

# Answer function with synonym and fuzzy matching
def answer_question_nlp(query, sections, qa_mapping, synonyms):
    section_key = match_with_synonyms(query, qa_mapping, synonyms)
    
    if section_key:
        # Retrieve the relevant section from the specified chapter
        return sections.get(section_key, "Section not found.")
    
    return "Sorry, I couldn't find an answer to your question."

# Example
user_query = "What is responsibility?"
answer = answer_question_nlp(user_query, sections, qa_mapping, synonyms)
print(answer)

Processed Query: responsibility
Corrected Query: responsibility
Checking key: responsibility, value: responsibility
Trying synonym: responsibility
Match found with synonym: responsibility
Responsibilities of leadership.
73. (1) Authority assigned to a State officer—
(a) is a public trust to be exercised in a manner that—
(i) is consistent with the purposes and objects of this
Constitution;
(ii) demonstrates respect for the people;
(iii) brings honour to the nation and dignity to the office; and
(iv) promotes public confidence in the integrity of the office;
and
(b) vests in the State officer the responsibility to serve the people,
rather than the power to rule them.
(2) The guiding principles of leadership and integrity include—
(a) selection on the basis of personal integrity, competence and
suitability, or election in free and fair elections;
(b) objectivity and impartiality in decision making, and in ensuring
that decisions are not influenced by nepotism, favouritism,
other improper

In [31]:
# Define the synonyms explicitly so that we can easily reference them later
synonyms = {
    "supremacy": ["supremacy", "authority", "ultimate power"],
    "sovereignty": ["sovereignty", "power of the people", "authority of the people", "self rule", "autonomy"],
    "defence": ["defense", "protection", "preservation"],
    "declaration": ["declaration", "proclamation", "statement", "announcement", "affirmation"],
    "territory": ["territory", "land", "region", "area", "jurisdiction", "bounds"],
    "devolution": ["devolution", "decentralization", "delegation", "transfer of power", "local governance", "subsidiarity"],
    "languages": ["languages", "tongues", "dialects", "official languages", "linguistic diversity"],
    "religion": ["religion", "faith", "belief systems", "spiritual practice", "secularism", "church-state separation"],
    "symbol": ["symbol", "emblem", "insignia", "representation", "national icon"],
    "day": ["day", "holiday", "observance", "public holiday", "commemoration", "remembrance"],
    "value": ["value", "principle", "ethic", "core value", "standard", "national ideal"],
    "governance": ["governance", "government", "administration", "management", "public service", "political structure"],
    "culture": ["culture", "heritage", "tradition", "customs", "societal norms", "arts"],
    "entitlement": ["entitlement", "right", "eligibility", "entitlement rights", "benefits", "privileges"],
    "retention": ["retention", "maintenance", "keeping", "preservation", "continuation"],
    "birth": ["birth", "nativity", "origin", "ancestry", "inborn citizenship"],
    "registration": ["registration", "enlistment", "enrollment", "citizenship application", "naturalization"],
    "dual": ["dual", "multiple", "dual nationality", "two-fold citizenship"],
    "revocation": ["revocation", "cancellation", "annulment", "rescission", "forfeiture", "withdrawal"],
    "legislation": ["legislation", "laws", "legal framework", "statutes", "enactment"],
    "fundamental": ["fundamental", "fundamental freedoms", "essential"],
    "application": ["application", "exercise", "freedom enforcement"],
    "implementation": ["implementation", "right support", "execution"],
    "authority": ["authority", "judicial power", "court jurisdiction", "judicial review"],
    "limitation": ["limitation", "freedom restriction", "limit entitlement"],
    "limit": ["limit","absolute right", "inalienable freedom", "immutable right"],
    "life": ["life", "existence", "entitlement to life", "survive"],
    "equality": ["equality", "equal treatment", "justice"],
    "dignity": ["dignity", "intrinsic respect", "inherent value", "personal honor"],
    "security": ["security", "liberty", "personal safety"],
    "slavery": ["slavery", "bondage", "force servitude", "involuntary labor", "coerce work"],
    "privacy": ["privacy", "confidential", "personal space"],
    "conscience": ["conscience", "liberty of thought", "religion freedom", "belief"],
    "expression": ["expression", "communication", "speak"],
    "medium": ["medium", "press autonomy", "journalist freedom", "right to information"],
    "information": ["information", "transparent", "datum"],
    "association": ["association", "union", "group formation", "association right"],
    "assembly": ["assembly", "dissent", "peace assembly", "demonstrate", "petition right"],
    "political": ["political", "electoral right", "political engagement", "vote right"],
    "movement": ["movement", "migration", "resident freedom", "movement liberty"],
    "Property": ["Property", "property safeguard", "assets security"],
    "work": ["work", "benefit", "employ"],
    "environment": ["environment", "conserve", "right to a healthy environment"],
    "economic": ["economic", "standard of life", "social welfare"],
    "language": ["language", "cultural right", "linguistic freedom", "cultural expression"],
    "family": ["family", "society foundation", "parent right"],
    "consumer": ["consumer", "client entitlement", "buyer safeguard"],
    "administrative": ["administrative", "fair administration rule", "legal administration measure"],
    "justice": ["justice", "legal access", "justice available"],
    "arrest": ["arrest", "capture", "seizure", "apprehension"],
    "fair hearing": ["fair hearing", "prosecute", "equity hearing"],
    "custody": ["custody", "confinement", "imprisonment", "prison"],
    "interpret": ["interpret", "explain", "clarify", "overview"],
    "infant": ["infant", "child", "kid", "toddler"],
    "disable": ["disable", "handicap", "impairment", "challenge"],
    "youth": ["youth", "young adult", "adolescent"],
    "minority": ["minority", "diversity right", "affirmative right"],
    "old": ["old", "elder", "vintage"],
    "emergency": ["emergency", "danger", "crisis", "disaster"],
    "national": ["national", "human right", "equaity Body", "right authority"],
    "principle": ["principle", "land management principle", "land policy guide"],
    "classification": ["classification", "land category", "types of land"],
    "public": ["public", "government land", "state property", "national land", "public landhold"],
    "community": ["community", "ethnic land", "cultural landhold", "community land"],
    "private": ["private", "individual landhold", "personal property", "freehold land", "private land"],
    "landhold" : ["landhold", "foreign lease", "alien land", "non-citizen land"],
    "regulation" : ["regulation", "land use policy", "property regulation", "land oversight"],
    "commission" : ["commission", "land authority", "public land commission", "land policy agency"],
    "land" : ["land", "land legislation", "property law", "land-use regulation", "land act"],
    "obligation" : ["obligation", "biodiversity duty", "conservation mandate"],
    "environmental" : ["environmental", "ecological redress", "sustainability enforcement"],
    "resource" : ["resource", "resource use agreement", "environmental concession"],
    "legislation" : ["legislation", "environmental law", "green policy enactment"],
    "responsibility": ["responsibility", "accountable", "commitment", "charge"],
    "oath": ["oath", "swear", "state office affirmation", "vow", "declare"],
    "conduct": ["conduct", "behavior", "comportment", "demeanor"],
    "finance": ["finance", "finance integrity", "wealth", "invest", "finance management"],
    "restrict": ["restrict", "constrain", "regulate", "prohibit", "curtail", "control"],
    "citizen": ["citizen", "inhabitant", "local resident"],
    "establish": ["establish", "corrupt", "bribe", "fraud"],
    "leader": ["leader", "governing, authority"]

}

# Define qa mapping explicitly too
qa_mapping = {
    "supremacy": "supremacy",
    "sovereignty": "sovereignty",
    "defence": "defence",
    "declaration": "declaration",
    "territory": "territory", 
    "devolution": "devolution",
    "language": "languages",
    "religion": "religion",
    "symbol": "symbol",
    "day": "day",
    "value": "value",
    "culture": "culture",
    "entitlement": "entitlement",
    "retention": "retention",  
    "birth": "birth",
    "registration": "registration",
    "dual": "dual",
    "revocation": "revocation",
    "legislation": "legislation",
    "fundamental": "fundamental",
    "application": "application",
    "implementation": "implementation",
    "authority": "authority",
    "limitation": "limitation",
    "limit": "limit",
    "life": "life",
    "equality": "equality",
    "dignity": "dignity",
    "security": "security",
    "slavery": "slavery",
    "privacy": "privacy",
    "conscience": "conscience",
    "expression": "expression",
    "medium": "medium",
    "information": "information",
    "association": "association",
    "assembly": "assembly",
    "political": "political",
    "movement": "movement",
    "property": "property",
    "work": "work",
    "environment": "environment",
    "economic": "economic",
    "language": "language",
    "family": "family",
    "consumer": "consumer",
    "administrative": "administrative",
    "justice": "justice",
    "arrest": "arrest",
    "fair hearing": "fair hearing",
    "custody": "custody",
    "interpret": "interpret",
    "infant": "infant",
    "disable": "disable",
    "youth": "youth",
    "minority": "minority",
    "old": "old",
    "emergency": "emergency",
    "national": "national",
    "principle": "principle",
    "classification": "classification",
    "public": "public",
    "community": "community",
    "private": "private",
    "landhold": "landhold",
    "regulation": "regulation",
    "commission": "commission",
    "land": "land",
    "obligation": "obligation",
    "environmental": "environmental",
    "resource": "resource",
    "legislation": "legislation",
    "responsibility": "responsibility",
    "oath": "oath",
    "conduct": "conduct",
    "finance": "finance",
    "restrict": "restrict",
    "citizen": "citizen",
    "establish": "establish",
    "leader": "leader"
    
}

combined_sections = {**chapter_1_sections, **chapter_2_sections, **chapter_3_sections, **chapter_4_sections, **chapter_5_sections, **chapter_6_sections }
# Print the keys of the dictionary
combined_sections.keys()

dict_keys(['sovereignty', 'supremacy', 'defence', 'declaration', 'territory', 'devolution', 'language', 'religion', 'symbol', 'day', 'value', 'culture', 'entitlement', 'retention', 'birth', 'registration', 'dual', 'revocation', 'legislation', 'fundamental', 'application', 'implementation', 'enforcement', 'authority', 'limitation', 'limit', 'life', 'equality', 'dignity', 'security', 'slavery', 'privacy', 'conscience', 'expression', 'medium', 'information', 'association', 'assembly', 'political', 'movement', 'property', 'work', 'environment', 'economic', 'family', 'consumer', 'administrative', 'justice', 'arrest', 'fair hearing', 'custody', 'interpret', 'infant', 'disable', 'youth', 'minority', 'old', 'emergency', 'national', 'principle', 'classification', 'public', 'community', 'private', 'landhold', 'regulation', 'commission', 'land', 'obligation', 'environmental', 'resource', 'responsibility', 'oath', 'conduct', 'finance', 'restrict', 'citizen', 'establish', 'leader'])

# CHAPTER 7-9 JASMINE!

# *****************

# CHAPTER 10

In [32]:
# Extract chapter 10
chapter_10 = extract_specific_pages(pdf_path, 94,106 )
chapter_10_trimmed= chapter_10.split("CHAPTER TEN")[1].strip()

In [33]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Preprocess the user query using spaCy
def preprocess_query(query):
    # Parse the query with spaCy
    doc = nlp(query)
    # Normalize the query: lowercase, lemmatize, and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Example usage of the preprocessing function
user_query = "What is Judicial Authority and Legal System?"
processed_query = preprocess_query(user_query)
print(processed_query)

judicial authority legal system


In [34]:
def split_chapter(chapter_text):
    # Split at key headings and strip extra whitespace
    sections = {
        "judicial authority": [],
        "judicial independence": [],
        "judicial offices": [], 
        "court systems": [],
        "supreme court": [],
        "appeal court": [],
        "high court" : [],
        "judicial appointments": [],
        "office tenure": [],
        "office removal": [],
        "subordinate courts" : [],
        "jsc establishment": [],
        "jsc functions": [],
        "judiciary fund": [] 
    }

    # Split by new lines to process line by line
    lines = chapter_text.splitlines()

    current_section = None

    for line in lines:
        stripped_line = line.strip()
        

        if stripped_line.startswith("Judicial authority"):
            current_section = "judicial authority"
        elif stripped_line.startswith("Independence of the Judiciary"):
            current_section = "judicial independence"
        elif stripped_line.startswith("Judicial offices and officers"):
            current_section = "judicial offices"
        elif stripped_line.startswith("System of courts"):
            current_section = "court systems"
        elif stripped_line.startswith("Supreme Court"):
            current_section = "supreme court"
        elif stripped_line.startswith("Court of Appeal"):
            current_section = "appeal court"
        elif stripped_line.startswith("High Court"):
            current_section = "high court"
        elif ("Appointment" in stripped_line and 
            ("Chief Justice" in stripped_line or "Deputy Chief Justice" in stripped_line or "judges" in stripped_line)):
             current_section = "judicial appointments"
        elif stripped_line.startswith("Tenure of office of the Chief Justice and other judges"):
            current_section = "office tenure"
        elif stripped_line.startswith("Removal from office"):
            current_section = "office removal"
        elif stripped_line.startswith("Subordinate courts"):
            current_section = "subordinate courts"
        elif stripped_line.startswith("Establishment of the Judicial Service Commission"):
            current_section = "jsc establishment"
        elif stripped_line.startswith("Functions of the Judicial Service Commission"):
            current_section = "jsc functions"
        elif stripped_line.startswith("Judiciary Fund"):
            current_section = "judiciary fund"  

        # Append line to the current section if it's set
        if current_section:
            sections[current_section].append(stripped_line)

    # Join each section into a single string
    for key in sections:
        sections[key]= "\n".join(sections[key])

    return sections

# Split the chapter into sections
chapter_10_sections= split_chapter(chapter_10_trimmed)

# Print a specific section as an example
print("judicial authority Section:\n", chapter_10_sections['judicial authority'])

judicial authority Section:
 Judicial authority.
159. (1) Judicial authority is derived from the people and vests in,
and shall be exercised by, the courts and tribunals established by or
under this Constitution.
(2) In exercising judicial authority, the courts and tribunals shall
be guided by the following principles—
(a) justice shall be done to all, irrespective of status;
(b) justice shall not be delayed;
(c) alternative forms of dispute resolution including reconciliation,
96 Constitution of Kenya, 2010
mediation, arbitration and traditional dispute resolution
mechanisms shall be promoted, subject to clause (3);
(d) justice shall be administered without undue regard to
procedural technicalities; and
(e) the purpose and principles of this Constitution shall be
protected and promoted.
(3) Traditional dispute resolution mechanisms shall not be used
in a way that—
(a) contravenes the Bill of Rights;
(b) is repugnant to justice and morality or results in outcomes that
are repugnant to 

## Question Answering mechanism 

In [35]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Preprocess the user query using spaCy
def preprocess_query(query):
    # Parse the query with spaCy
    doc = nlp(query)
    # Normalize the query: lowercase, lemmatize, and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Define the sections variable with Chapter 10
sections = chapter_10_sections

# Define the QA mapping based on key phrases and corresponding sections
qa_mapping = {
    "judicial authority": "judicial authority",
    "judicial independence": "judicial independence",
    "judicial office": "judicial offices",
    "system": "court systems",
    "supreme court": "supreme court",
    "appeal court": "appeal court",
    "high court": "high court",
    "judicial appointment": "judicial appointments",
    "office tenure": "office tenure",
    "office removal": "office removal",
    "subordinate court": "subordinate courts",
    "jsc establishment": "jsc establishment",
    "jsc function": "jsc functions",
    "judiciary fund": "judiciary fund"
}


# Update the Q&A system to use preprocessed queries
def answer_question_nlp(query, sections, qa_mapping):
    # Preprocess the user query
    processed_query= preprocess_query(query)
    # Debug
    print( f"Processed query:{processed_query}")
    
    # Search for a key in qa_mapping that matches the preprocessed query
    for key in qa_mapping:
        if key in processed_query:
            # Return the relevant section in the text
            return sections[qa_mapping[key]]
    
        
    return "Sorry, I couldn't find an answer to your question."

# Example 1
user_query = "what is the judiciary fund?"
answer = answer_question_nlp(user_query, sections, qa_mapping)
print(answer)

Processed query:judiciary fund
Judiciary Fund.
173. (1) There is established a fund to be known as the Judiciary
Fund which shall be administered by the Chief Registrar of the
Judiciary.
(2) The Fund shall be used for administrative expenses of the
Judiciary and such other purposes as may be necessary for the
discharge of the functions of the Judiciary.
(3) Each financial year, the Chief Registrar shall prepare
estimates of expenditure for the following year, and submit them to the
National Assembly for approval.
(4) On approval of the estimates by the National Assembly, the
expenditure of the Judiciary shall be a charge on the Consolidated
Fund and the funds shall be paid directly into the Judiciary Fund.
(5) Parliament shall enact legislation to provide for the regulation
of the Fund.


In [36]:
# Example 2
user_query = "what is office tenure?"
answer = answer_question_nlp(user_query, sections, qa_mapping)
print(answer)

Processed query:office tenure
Tenure of office of the Chief Justice and other judges.
167. (1) A judge shall retire from office on attaining the age of
seventy years, but may elect to retire at any time after attaining the
age of sixty-five years.
(2) The Chief Justice shall hold office for a maximum of ten years
or until retiring under clause (1), whichever is the earlier.
(3) If the Chief Justice’s term of office expires before the Chief
Justice retires under clause (1), the Chief Justice may continue in
102 Constitution of Kenya, 2010
office as a judge of the Supreme Court.
(4) If, on the expiry of the term of office of a Chief Justice, the
Chief Justice opts to remain on the Supreme Court under clause (3),
the next person appointed as Chief Justice may be selected in
accordance with Article 166 (1), even though that appointment may
result in there being more than the maximum permitted number of


### Chapter 10 NLU

In [37]:
# Define synonym mapping
synonyms_1 = {
    "judicial authority": ["judicial authority","legal authority","court jurisdiction"],
    "judicial independence": ["judicial independence","judicial autonomy","judicial impartiality"],
    "judicial offices": ["judicial office","legal position","judicial role"],
    "court systems": ["court system","judicial system","legal system"],
    "supreme court": ["supreme court","highest court","apex court"],
    "appeal court": ["appeal court","appellate court","review court"],
    "high court": ["high court","superior court"],
    "judicial appointments": ["judicial appointment","judge appointment","judicial selection"],
    "office tenure": ["office tenure","office term","service duration"],
    "office removal": ["office removal","office dismissal","office termination"],
    "subordinate courts": ["subordinate court","local court","inferior court"],
    "jsc establishment": ["judicial service commission establishment","judicial service commission creation","judicial service commission formation"],
    "jsc functions": ["judicial service commission function","judicial service commission role","judicial service commission responsibility"],
    "judiciary fund": ["judiciary fund","judicial fund","legal fund"]

  
}
# QA mapping
qa_mapping = {
    "judicial authority": "judicial authority",
    "judicial independence" :"judiciary independence",
    "judicial offices": "judicial offices",
    "court systems": "court systems",
    "supreme court": "supreme court",
    "appeal court": "appeal court",
    "high court": "high court",
    "judicial appointments": "judicial appointments",
    "office tenure": "office tenure",
    "office removal": "office removal",
    "subordinate courts": "subordinate courts",
    "jsc establishment": "jsc establishment",
    "jsc functions": "jsc functions",
    "judiciary fund": "judiciary fund"
}

sections = chapter_10_sections

# Preprocess the query
def preprocess_query(query):
    doc = nlp(query)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Function to correct the spelling
def correct_spelling(processed_query):
    spell = SpellChecker()
    words = processed_query.split()
    # Find misspelled words
    misspelled_words = spell.unknown(words)


    corrected_words = []
    for word in words:
        # Correct the word if it's misspelled
        if word in misspelled_words:
            corrected_word = spell.correction(word) # Get the most likely correction
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)

    # Reconstruct the initial sentence
    corrected_input = " ".join(corrected_words)

    return corrected_input

            

# Match with synonym support
def match_with_synonyms(query, qa_mapping, synonyms_1):
    processed_query = preprocess_query(query)
    print(f"Processed Query: {processed_query}")  

    # Correct spelling in the processed query
    corrected_query = correct_spelling(processed_query)
    print(f"Corrected Query: {corrected_query}")  


    for key, value in qa_mapping.items():
        print(f"Checking key: {key}, value: {value}")  
        for synonym in synonyms_1.get(key, [key]):
            print(f"Trying synonym: {synonym}")  
            if synonym in corrected_query:
                print(f"Match found with synonym: {synonym}")  
                return key  # Only return section value of the key
    
    print("No match found")  
    return None

# Answer function with synonym and fuzzy matching
def answer_question_nlp(query, sections, qa_mapping, synonyms):
    section_key = match_with_synonyms(query, qa_mapping, synonyms)
    
    if section_key:
        # Retrieve the relevant section from the specified chapter
        return sections.get(section_key, "Section not found.")
    
    return "Sorry, I couldn't find an answer to your question."

# Example 1
user_query = "What is court jurisdiction?"
answer = answer_question_nlp(user_query, sections, qa_mapping, synonyms_1)
print(answer)



Processed Query: court jurisdiction
Corrected Query: court jurisdiction
Checking key: judicial authority, value: judicial authority
Trying synonym: judicial authority
Trying synonym: legal authority
Trying synonym: court jurisdiction
Match found with synonym: court jurisdiction
Judicial authority.
159. (1) Judicial authority is derived from the people and vests in,
and shall be exercised by, the courts and tribunals established by or
under this Constitution.
(2) In exercising judicial authority, the courts and tribunals shall
be guided by the following principles—
(a) justice shall be done to all, irrespective of status;
(b) justice shall not be delayed;
(c) alternative forms of dispute resolution including reconciliation,
96 Constitution of Kenya, 2010
mediation, arbitration and traditional dispute resolution
mechanisms shall be promoted, subject to clause (3);
(d) justice shall be administered without undue regard to
procedural technicalities; and
(e) the purpose and principles of th

In [38]:
# Example 2
user_query2 = " what is process for judge appointments?"
answer = answer_question_nlp(user_query2, sections, qa_mapping, synonyms_1)
print(answer)

Processed Query:   process judge appointment
Corrected Query: process judge appointment
Checking key: judicial authority, value: judicial authority
Trying synonym: judicial authority
Trying synonym: legal authority
Trying synonym: court jurisdiction
Checking key: judicial independence, value: judiciary independence
Trying synonym: judicial independence
Trying synonym: judicial autonomy
Trying synonym: judicial impartiality
Checking key: judicial offices, value: judicial offices
Trying synonym: judicial office
Trying synonym: legal position
Trying synonym: judicial role
Checking key: court systems, value: court systems
Trying synonym: court system
Trying synonym: judicial system
Trying synonym: legal system
Checking key: supreme court, value: supreme court
Trying synonym: supreme court
Trying synonym: highest court
Trying synonym: apex court
Checking key: appeal court, value: appeal court
Trying synonym: appeal court
Trying synonym: appellate court
Trying synonym: review court
Checking 

# Chapter 11

In [39]:
# chapter 11 extraction
chapter_11= extract_specific_pages(pdf_path, 106,121)
chapter_11_trimmed = chapter_11.split("CHAPTER ELEVEN")[1].strip()
chapter_11_trimmed = chapter_11.split("CHAPTER TWELVE")[0].strip()

In [40]:
def split_chapter(chapter_text):
    # Split at key headings and strip extra whitespace
    sections = {
        "object devolution": [],
        "principles of devolved government": [],
        "county governments": [],
        "membership": [],
        "speaker": [],
        "county executive committees": [],
        "county election": [],
        "county removal": [],
        "vacancy": [],
        "functions of county executive committees": [],
        "cities": [],
        "legislative authority": [],
        "functions and powers of governments": [],
        "transfer": [],
        "boundaries of counties": [],
        "cooperation": [],
        "support": [],
        "conflict of laws": [],
        "suspension": [],
        "qualifications": [],
        "vacation": [],
        "summon witnesses": [],
        "public participation and county assembly powers, privileges and immunities": [],
        "gender balance": [],
        "county transition": [],
        "publication": [],
        "legislation on chapter": []
    }

    # Split by new lines to process line by line
    lines = chapter_text.splitlines()

    current_section = None

    for line in lines:
        stripped_line = line.strip()

        if stripped_line.startswith("Objects of devolution"):
            current_section = "object devolution"
        elif stripped_line.startswith("Principles of devolved government"):
            current_section = "principles of devolved government"
        elif stripped_line.startswith("County governments"):
            current_section = "county governments"
        elif stripped_line.startswith("Membership of county assembly"):
            current_section = "membership"
        elif stripped_line.startswith("Speaker of a county assembly"):
            current_section = "speaker"
        elif stripped_line.startswith("County executive committees"):
            current_section = "county executive committees"
        elif stripped_line.startswith("Election of county governor and deputy county governor"):
            current_section = "county election"
        elif stripped_line.startswith("Removal of a county governor"):
            current_section = "county removal"
        elif stripped_line.startswith("Vacancy in the office of county governor"):
            current_section = "vacancy"
        elif stripped_line.startswith("Functions of county executive committees"):
            current_section = "functions of county executive committees"
        elif stripped_line.startswith("Urban areas and cities"):
            current_section = "cities"
        elif stripped_line.startswith("Legislative authority of county assemblies"):
            current_section = "legislative authority"
        elif stripped_line.startswith("Respective functions and powers of national and county governments"):
            current_section = "functions and powers of governments"
        elif stripped_line.startswith("Transfer of functions and powers between levels of government"):
            current_section = "transfer"
        elif stripped_line.startswith("Boundaries of counties"):
            current_section = "boundaries of counties"
        elif stripped_line.startswith("Cooperation between national and county governments"):
            current_section = "cooperation"
        elif stripped_line.startswith("Support for county governments"):
            current_section = "support"
        elif stripped_line.startswith("Conflict of laws"):
            current_section = "conflict of laws"
        elif stripped_line.startswith("Suspension of county government"):
            current_section = "suspension"
        elif stripped_line.startswith("Qualifications for election as member of county assembly"):
            current_section = "qualifications"
        elif stripped_line.startswith("Vacation of office of member of county assembly"):
            current_section = "vacation"
        elif stripped_line.startswith("County assembly power to summon witnesses"):
            current_section = "summon witnesses"
        elif stripped_line.startswith("Public participation and county assembly powers, privileges and immunities"):
            current_section = "public participation and county assembly powers, privileges and immunities"
        elif stripped_line.startswith("County assembly gender balance and diversity"):
            current_section = "gender balance"
        elif stripped_line.startswith("County government during transition"):
            current_section = "county transition"
        elif stripped_line.startswith("Publication of county legislation"):
            current_section = "publication"
        elif stripped_line.startswith("Legislation on Chapter"):
            current_section = "legislation on chapter"
        # Append line to the current section if it's set
        if current_section:
            sections[current_section].append(stripped_line)

    # Join each section into a single string
    for key in sections:
        sections[key] = "\n".join(sections[key])

    return sections



# Split the chapter into sections
chapter_11_sections = split_chapter(chapter_11_trimmed)

# Print a specific section as an example
print("\n publication Section:\n", chapter_11_sections["publication"])



 publication Section:
 Publication of county legislation.
199. (1) County legislation does not take effect unless published
in the Gazette.
(2) National and county legislation may prescribe additional
requirements in respect of the publication of county legislation.
Constitution of Kenya, 2010 121


## Question Answering mechanism 

In [41]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Preprocess the user query using spaCy
def preprocess_query(query):
    # Parse the query with spaCy
    doc = nlp(query)
    # Normalize the query: lowercase, lemmatize, and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Define the sections variable with Chapter 2
sections = chapter_11_sections

# Define the QA mapping based on key phrases and corresponding sections
qa_mapping = {
    "object devolution": "object devolution",
    "devolve government": "principles of devolved government",
    "county government": "county governments",
    "membership": "membership",
    "speaker": "speaker",
    "county executive committee": "county executive committees",
    "county election": "county election",
    "county removal": "county removal",
    "vacancy": "vacancy",
    "executive function": "functions of county executive committees",
    "city": "cities",
    "legislative authority": "legislative authority",
      "function and power of national and county government":"respective functions and powers of national and county governments.",
    "transfer": "transfer",
    "county boundary": "boundaries of counties",
    "cooperation": "cooperation",
    "support": "support",
    "law conflict": "conflict of laws",
    "suspension county government": "suspension of county government",
    "qualification": "qualifications",
    "vacation": "vacation",
    "summon witness": "summon witnesses",
    "public participation county assembly power privilege immunity": "public participation and county assembly powers, privileges and immunities",
    "gender balance": "gender balance",
    "transition": "county transition",
    "publication": "publication",
    "legislation": "legislation on chapter"
}

# Update the Q&A system to use preprocessed queries
def answer_question_nlp(query, sections, qa_mapping):
    # Preprocess the user query
    processed_query = preprocess_query(query)

    # Debug
    print(f"Processed query: {processed_query}")
    
    # Search for a key in qa_mapping that matches the preprocessed query
    for key in qa_mapping:
        if key in processed_query:
            # Return the relevant section in the text
            return sections[qa_mapping[key]]
            
    return "Sorry, I couldn't find an answer to your question."


# Example 1
user_query = "membership?"
answer = answer_question_nlp(user_query, sections, qa_mapping)
print(answer)

Processed query: membership
Membership of county assembly.
177. (1) A county assembly consists of—
(a) members elected by the registered voters of the wards, each
ward constituting a single member constituency, on the same
day as a general election of Members of Parliament, being
the second Tuesday in August, in every fifth year;
(b) the number of special seat members necessary to ensure that
no more than two-thirds of the membership of the assembly
are of the same gender;
(c) the number of members of marginalised groups, including
persons with disabilities and the youth, prescribed by an Act
of Parliament; and
(d) the Speaker, who is an ex officio member.
(2) The members contemplated in clause (1) (b) and (c) shall, in
each case, be nominated by political parties in proportion to the seats
received in that election in that county by each political party under
paragraph (a) in accordance with Article 90.
(3) The filling of special seats under clause (1) (b) shall be
determined after de

In [42]:
user_query = "what is gender balance?"
answer= answer_question_nlp(user_query, sections, qa_mapping)
print(answer)

Processed query: gender balance
County assembly gender balance and diversity.
197. (1) Not more than two-thirds of the members of any county
assembly or county executive committee shall be of the same gender.
(2) Parliament shall enact legislation to—
(a) ensure that the community and cultural diversity of a county is
reflected in its county assembly and county executive
committee; and
(b) prescribe mechanisms to protect minorities within counties.


## Chapter 11 NLU

In [43]:
# Define synonym mapping
synonyms_2 = {
    "object devolution": ["object devolution", "power decentralization", "government devolution"],
    "devolve government": ["devolve government", "decentralization principle"],
    "county government": ["county government", "local government", "regional administration"],
    "membership": ["membership", "composition", "structure"],
    "speaker": ["speaker", "assembly speaker", "preside officer"],
    "county executive committee": ["county executive committee", "regional executive body", "local executive board"],
    "county election": ["county election", "local election", "regional election"],
    "county removal": ["county removal", "county office dismissal", "regional removal"],
    "vacancy": ["vacancy", "position vacancy", "office vacancy"],
    "functions of county executive committees": ["functions of county executive committees", "duties of county executive commitees", "county executive commitees roles"],
    "city": ["city", "urban center", "municipality"],
    "legislative authority": ["legislative authority", "law make power", "regulatory authority"],
     "function and power of national and county government": ["function and power of national and county government", "government duty", "government responsibility"],
    "transfer": ["transfer", "delegation", "reassignment"],
    "county boundary": ["county boundary", "county border", "regional limit"],
    "cooperation": ["cooperation", "collaboration", "coordination"],
    "support": ["support", "assistance", "aid"],
    "conflict of law": ["conflict of law", "legal discrepancy", "jurisdiction conflict"],
      "suspension": ["suspension", "interruption", "temporary halt"],
    "qualification": ["qualification", "eligibility", "requirement"],
    "vacation": ["vacation", "office leave", "office absence"],
    "summon witness": ["summon witness", "convene witness", "request witness attendance"],
    "public participation county assembly power privilege immunity": ["public participation county assembly power privilege immunity", "county assembly right", "local governance privilege"],
    "gender balance": ["gender balance", "gender equity", "equal representation"],
    "county transition": ["county transition", "regional handover", "local government transition"],
    "publication": ["publication", "official release", "public announcement"],
    "legislation": ["legislation", "regulation", "law"]
    
}

# QA mapping
qa_mapping = {
    "object devolution": "object devolution",
    "devolve government": "devolve government",
    "county government": "county government",
    "membership": "membership",
    "speaker": "speaker",
    "county executive committee": "county executive committee",
    "county election": "county election",
    "county removal": "county removal",
    "vacancy": "vacancy",
    "functions of county executive committees": "executive function",
    "city": "city",
    "legislative authority": "legislative authority",
    "function and power of national and county government": "function and power of national and county government",
    "transfer": "transfer",
    "county boundary": "county boundary",
    "cooperation": "cooperation",
    "support": "support",
    "conflict of law": "law conflict",
    "suspension": "suspension county government",
    "qualification": "qualification",
    "vacation": "vacation",
    "summon witness": "summon witness",
    "public participation county assembly power privilege immunity": "public participation county assembly power privilege immunity",
    "gender balance": "gender balance",
    "county transition": "transition",
    "publication": "publication",
    "legislation": "legislation"
  
}


sections = chapter_11_sections

# Preprocess the query
def preprocess_query(query):
    doc = nlp(query)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Function to correct the spelling
def correct_spelling(processed_query):
    spell = SpellChecker()
    words = processed_query.split()
    # Find misspelled words
    misspelled_words = spell.unknown(words)


    corrected_words = []
    for word in words:
        # Correct the word if it's misspelled
        if word in misspelled_words:
            corrected_word = spell.correction(word) # Get the most likely correction
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)

    # Reconstruct the initial sentence
    corrected_input = " ".join(corrected_words)

    return corrected_input

# Match with synonym support
def match_with_synonyms(query, qa_mapping, synonyms_2):
    processed_query = preprocess_query(query)
    print(f"Processed Query: {processed_query}")  

    # Correct spelling in the processed query
    corrected_query = correct_spelling(processed_query)
    print(f"Corrected Query: {corrected_query}")  


    for key, value in qa_mapping.items():
        print(f"Checking key: {key}, value: {value}")  
        for synonym in synonyms_2.get(key, [key]):
            print(f"Trying synonym: {synonym}")  
            if synonym in corrected_query:
                print(f"Match found with synonym: {synonym}")  
                return key  # Only return section value of the key
    
    print("No match found")  
    return None

# Answer function with synonym and fuzzy matching
def answer_question_nlp(query, sections, qa_mapping, synonyms):
    section_key = match_with_synonyms(query, qa_mapping, synonyms)
    
    if section_key:
         # Retrieve the relevant section from the specified chapter
        return sections.get(section_key, "Section not found.")
    
    return "Sorry, I couldn't find an answer to your question."

# Example 1
user_query = "steps when vacancy in office occurs?"
answer = answer_question_nlp(user_query, sections, qa_mapping, synonyms_2)
print(answer)


Processed Query: step vacancy office occur
Corrected Query: step vacancy office occur
Checking key: object devolution, value: object devolution
Trying synonym: object devolution
Trying synonym: power decentralization
Trying synonym: government devolution
Checking key: devolve government, value: devolve government
Trying synonym: devolve government
Trying synonym: decentralization principle
Checking key: county government, value: county government
Trying synonym: county government
Trying synonym: local government
Trying synonym: regional administration
Checking key: membership, value: membership
Trying synonym: membership
Trying synonym: composition
Trying synonym: structure
Checking key: speaker, value: speaker
Trying synonym: speaker
Trying synonym: assembly speaker
Trying synonym: preside officer
Checking key: county executive committee, value: county executive committee
Trying synonym: county executive committee
Trying synonym: regional executive body
Trying synonym: local executive

# Chapter 12

In [44]:
chapter_12= extract_specific_pages(pdf_path, 120,138)
chapter_12_trimmed= chapter_12.split("CHAPTER TWELVE")[1].strip()

In [45]:
def split_chapter_12(chapter_text):
    # Split at key headings and strip extra whitespace
    sections = {
        "principles of public finance": [],
        "equitable sharing of national revenue": [],
        "equitable share and other financial laws": [],
        "equalisation fund": [],
        "consultation on financial legislation": [],
        "consolidated fund": [],
        "revenue funds": [],
        "contingencies fund": [],
        "power to impose taxes": [],
        "imposition of tax": [],
        "borrowing by national government": [],
        "borrowing by counties": [],
        "loan guarantees": [],
        "public debt": [],
        "commission on revenue allocation": [],
        "functions of the commission": [],
        "division of revenue": [],
        "annual division and allocation": [],
        "transfer of equitable share": [],
        "budget form and content": [],
        "budget estimates": [],
        "expenditure before budget": [],
        "supplementary appropriation": [],
        "county appropriation bills": [],
        "financial control": [],
        "accounts and audit": [],
        "procurement of public goods": [],
        "controller of budget": [],
        "auditor-general": [],
        "salaries and remuneration commission": [],
        "central bank of kenya": []
    }

    # Split by new lines to process line by line
    lines = chapter_text.splitlines()

    current_section = None

    for line in lines:
        stripped_line = line.strip()

        if stripped_line.startswith("Principles of public finance"):
            current_section = "principles of public finance"
        elif stripped_line.startswith("Equitable sharing of national revenue"):
            current_section = "equitable sharing of national revenue"
        elif stripped_line.startswith("Equitable share and other financial laws"):
            current_section = "equitable share and other financial laws"
        elif stripped_line.startswith("Equalisation Fund"):
            current_section = "equalisation fund"
        elif stripped_line.startswith("Consultation on financial legislation affecting counties"):
            current_section = "consultation on financial legislation"
        elif stripped_line.startswith("Consolidated Fund and other public funds"):
            current_section = "consolidated fund"
        elif stripped_line.startswith("Revenue Funds for county governments"):
            current_section = "revenue funds"
        elif stripped_line.startswith("Contingencies Fund"):
            current_section = "contingencies fund"
        elif stripped_line.startswith("Power to impose taxes and charges"):
            current_section = "power to impose taxes"
        elif stripped_line.startswith("Imposition of tax"):
            current_section = "imposition of tax"
        elif stripped_line.startswith("Borrowing by national government"):
            current_section = "borrowing by national government"
        elif stripped_line.startswith("Borrowing by counties"):
            current_section = "borrowing by counties"
        elif stripped_line.startswith("Loan guarantees by national government"):
            current_section = "loan guarantees"
        elif stripped_line.startswith("Public debt"):
            current_section = "public debt"
        elif stripped_line.startswith("Commission on Revenue Allocation"):
            current_section = "commission on revenue allocation"
        elif stripped_line.startswith("Functions of the Commission on Revenue Allocation"):
            current_section = "functions of the commission"
        elif stripped_line.startswith("Division of revenue"):
            current_section = "division of revenue"
        elif stripped_line.startswith("Annual Division and Allocation of Revenue Bills"):
            current_section = "annual division and allocation"
        elif stripped_line.startswith("Transfer of equitable share"):
            current_section = "transfer of equitable share"
        elif stripped_line.startswith("Form, content and timing of budgets"):
            current_section = "budget form and content"
        elif stripped_line.startswith("Budget estimates and annual Appropriation Bill"):
            current_section = "budget estimates"
        elif stripped_line.startswith("Expenditure before annual budget is passed"):
            current_section = "expenditure before budget"
        elif stripped_line.startswith("Supplementary appropriation"):
            current_section = "supplementary appropriation"
        elif stripped_line.startswith("County appropriation Bills"):
            current_section = "county appropriation bills"
        elif stripped_line.startswith("Financial control"):
            current_section = "financial control"
        elif stripped_line.startswith("Accounts and audit of public entities"):
            current_section = "accounts and audit"
        elif stripped_line.startswith("Procurement of public goods and services"):
            current_section = "procurement of public goods"
        elif stripped_line.startswith("Controller of Budget"):
            current_section = "controller of budget"
        elif stripped_line.startswith("Auditor-General"):
            current_section = "auditor-general"
        elif stripped_line.startswith("Salaries and Remuneration Commission"):
            current_section = "salaries and remuneration commission"
        elif stripped_line.startswith("Central Bank of Kenya"):
            current_section = "central bank of kenya"

        # Append line to the current section if it's set
        if current_section:
            sections[current_section].append(stripped_line)

    # Join each section into a single string
    for key in sections:
        sections[key] = "\n".join(sections[key])

    return sections



# Split Chapter 12 into sections
chapter_12_sections = split_chapter_12(chapter_12_trimmed)

# Print a specific section as an example
print("\n Principles of Public Finance Section:\n", chapter_12_sections["principles of public finance"])



 Principles of Public Finance Section:
 Principles of public finance.
201. The following principles shall guide all aspects of public
finance in the Republic—
(a) there shall be openness and accountability, including public
participation in financial matters;
(b) the public finance system shall promote an equitable society,
and in particular—
(i) the burden of taxation shall be shared fairly;
(ii) revenue raised nationally shall be shared equitably
among national and county governments; and
(iii) expenditure shall promote the equitable development of
the country, including by making special provision for
marginalised groups and areas;
(c) the burdens and benefits of the use of resources and public
borrowing shall be shared equitably between present and
future generations;
122 Constitution of Kenya, 2010
(d) public money shall be used in a prudent and responsible way;
and
(e) financial management shall be responsible, and fiscal
reporting shall be clear.


### Question Answering Mechanism

In [46]:
# Load spacy model
nlp = spacy.load("en_core_web_sm")

# Preprocess the user query using spaCy
def preprocess_query(query):
    # Parse the query with spaCy
    doc = nlp(query)
    # Normalize the query: lowercase, lemmatize, and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Define the sections variable with Chapter 12
sections = chapter_12_sections
    
# Define the QA mapping based on key phrases and corresponding sections
qa_mapping = {
    "principle public finance": "principles of public finance",
    "equitable sharing of national revenue": "equitable sharing  of national revenue",
    "equitable share law":"equitable share and other financial laws",
    "equalisation": "equalisation fund",
    "consultation": "consultation on financial legislation",
    "consolidate": "consolidated fund",
    "revenue": "revenue funds",
    "contingency": "contingencies fund",
    "tax": "power to impose taxes",
    "borrow": "borrowing by national government",
    "loan": "loan guarantees",
    "debt": "public debt",
    "commission": "commission on revenue allocation",
    "division": "division of revenue",
    "budget": "budget form and content",
    "expenditure": "expenditure before budget",
    "supplementary": "supplementary appropriation",
    "control": "financial control",
    "procurement": "procurement of public goods",
    "budget controller": "controller of budget",
    "auditor": "auditor-general",
    "salary": "salaries and remuneration commission",
    "bank": "central bank of kenya"
}

# Update the Q&A system to use preprocessed queries
def answer_question_nlp(query, sections, qa_mapping):
    # Preprocess the user query
    processed_query = preprocess_query(query)

    # Debug
    print(f"Processed query: {processed_query}")
    
    # Search for a key in qa_mapping12 that matches the preprocessed query
    for key in qa_mapping:
        if key in processed_query:
            # Return the relevant section in the text
            return sections[qa_mapping[key]]
            
    return "Sorry, I couldn't find an answer to your question."

# Example 1
user_query = "What are loan guarantees?"
response = answer_question_nlp(user_query, sections, qa_mapping)
print(response)


Processed query: loan guarantee
Loan guarantees by national government.
213. (1) An Act of Parliament shall prescribe terms and
conditions under which the national government may guarantee loans.
(2) Within two months after the end of each financial year, the
national government shall publish a report on the guarantees that it
gave during that year.


In [47]:
# Example 2
user_query = "Explain division on public debt?"
answer = answer_question_nlp(user_query, sections, qa_mapping)
print(answer)

Processed query: explain division public debt
Public debt.
214. (1) The public debt is a charge on the Consolidated Fund,
but an Act of Parliament may provide for charging all or part of the
public debt to other public funds.
(2) For the purposes of this Article, “the public debt” means all
financial obligations attendant to loans raised or guaranteed and
securities issued or guaranteed by the national government.
PART 4—REVENUE ALLOCATION


In [48]:
# Define synonym mapping
synonyms_3 = {
    "principle public finance": ["principle public finance", "public finance guideline", "fundamental financial rule"],
    "equitable sharing of national revenue": ["equitable share of national revenue", "fair distribution of revenue"],
    "equitable share law": ["equitable share law", "financial law", "equitable distribution law"],
    "equalisation fund": ["equalisation fund", "equalization", "equal opportunity fund"],
    "consultation": ["consultation", "discussion", "advisement"],
    "consolidated fund": ["consolidated fund", "main fund", "central fund"],
    "revenue funds": ["revenue funds", "revenue", "income fund"],
    "contingencies fund": ["contingency fund", "reserve fund", "emergency fund"],
    "tax impose": ["tax impose", "tax authority", "tax power"],
    "national government borrow": ["national government borrow", "national debt", "federal borrow"],
    "loan guarantee": ["loan guarantee", "credit guarantee", "loan assurance"],
    "public debt": ["public debt", "national debt", "government debt"],
    "commission on revenue allocation": ["commission on revenue allocation", "allocation commission", "revenue allocation committee"],
    "division of revenue": ["division of revenue", "revenue distribution", "allocation of revenue"],
    "budget content": ["budget content", "budget structure", "budget framework"],
    "expenditure": ["expenditure", "cost", "expense"],
    "supplementary appropriation": ["supplementary appropriation", "additional appropriation", "extra allocation"],
    "financial control": ["financial control", "financial oversight", "budget control"],
    "procurement": ["procurement", "acquisition", "purchase"],
    "budget contoller": ["budget contoller", "cost controller"],
    "auditor-general": ["auditor general", "chief auditor", "head auditor"],
    "salary": ["salary","remuneration", "pay"],
    "central bank of kenya": ["central bank of kenya", "central bank", "central financial authority"]
}

# QA mapping
qa_mapping = {
    "principle public finance": "principle public finance",
    "equitable sharing of national revenue": "equitable sharing of national revenue",
    "equitable share law": "equitable share",
    "equalisation fund": "equalisation fund",
    "consultation": "consultation",
    "consolidated fund": "consolidate",
    "revenue funds": "revenue",
    "contingencies fund": "contingency",
    "tax impose": "tax",
    "national government borrow": "borrow",
    "loan guarantee": "loan",
    "public debt": "debt",
    "commission on revenue allocation": "commission",
    "division of revenue": "division",
    "budget content": "budget",
    "expenditure": "expenditure",
    "supplementary appropriation": "supplementary",
    "financial control": "control",
    "procurement": "procurement",
    "budget controller": "budget controller",
    "auditor-general": "auditor",
    "salary": "salary",
    "central bank of kenya": "bank"
}


sections = chapter_12_sections

# Preprocess the query
def preprocess_query(query):
    doc = nlp(query)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Function to correct the spelling
def correct_spelling(processed_query):
    spell = SpellChecker()
    words = processed_query.split()
    # Find misspelled words
    misspelled_words = spell.unknown(words)


    corrected_words = []
    for word in words:
        # Correct the word if it's misspelled
        if word in misspelled_words:
            corrected_word = spell.correction(word) # Get the most likely correction
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)

    # Reconstruct the initial sentence
    corrected_input = " ".join(corrected_words)

    return corrected_input

# Match with synonym support
def match_with_synonyms(query, qa_mapping, synonyms_3):
    processed_query = preprocess_query(query)
    print(f"Processed Query: {processed_query}")  

    # Correct spelling in the processed query
    corrected_query = correct_spelling(processed_query)
    print(f"Corrected Query: {corrected_query}")  


    for key, value in qa_mapping.items():
        print(f"Checking key: {key}, value: {value}")  
        for synonym in synonyms_3.get(key, [key]):
            print(f"Trying synonym: {synonym}")  
            if synonym in corrected_query:
                print(f"Match found with synonym: {synonym}")  
                return key  # Only return section value of the key
    
    print("No match found")  
    return None

# Answer function with synonym and fuzzy matching
def answer_question_nlp(query, sections, qa_mapping, synonyms):
    section_key = match_with_synonyms(query, qa_mapping, synonyms)
    
    if section_key:
         # Retrieve the relevant section from the specified chapter
        return sections.get(section_key, "Section not found.")
    
    return "Sorry, I couldn't find an answer to your question."

# Example 1
user_query = "what is the equilization fund?"
answer = answer_question_nlp(user_query, sections, qa_mapping, synonyms_3)
print(answer)


Processed Query: equilization fund
Corrected Query: equalization fund
Checking key: principle public finance, value: principle public finance
Trying synonym: principle public finance
Trying synonym: public finance guideline
Trying synonym: fundamental financial rule
Checking key: equitable sharing of national revenue, value: equitable sharing of national revenue
Trying synonym: equitable share of national revenue
Trying synonym: fair distribution of revenue
Checking key: equitable share law, value: equitable share
Trying synonym: equitable share law
Trying synonym: financial law
Trying synonym: equitable distribution law
Checking key: equalisation fund, value: equalisation fund
Trying synonym: equalisation fund
Trying synonym: equalization
Match found with synonym: equalization
Equalisation Fund.
204. (1) There is established an Equalisation Fund into which
shall be paid one half per cent of all the revenue collected by the
national government each year calculated on the basis of the m

In [49]:
# Example 1
user_query = "roles of Central bank?"
answer = answer_question_nlp(user_query, sections, qa_mapping, synonyms_3)
print(answer)

Processed Query: role central bank
Corrected Query: role central bank
Checking key: principle public finance, value: principle public finance
Trying synonym: principle public finance
Trying synonym: public finance guideline
Trying synonym: fundamental financial rule
Checking key: equitable sharing of national revenue, value: equitable sharing of national revenue
Trying synonym: equitable share of national revenue
Trying synonym: fair distribution of revenue
Checking key: equitable share law, value: equitable share
Trying synonym: equitable share law
Trying synonym: financial law
Trying synonym: equitable distribution law
Checking key: equalisation fund, value: equalisation fund
Trying synonym: equalisation fund
Trying synonym: equalization
Trying synonym: equal opportunity fund
Checking key: consultation, value: consultation
Trying synonym: consultation
Trying synonym: discussion
Trying synonym: advisement
Checking key: consolidated fund, value: consolidate
Trying synonym: consolidated

## Cumulative synonyms, qa_mapping and sections

In [50]:
# Define the synonyms explicitly so that we can easily reference them later
synonyms = {
    "supremacy": ["supremacy", "authority", "ultimate power"],
    "sovereignty": ["sovereignty", "power of the people", "authority of the people", "self rule", "autonomy"],
    "defence": ["defense", "protection", "preservation"],
    "declaration": ["declaration", "proclamation", "statement", "announcement", "affirmation"],
    "territory": ["territory", "land", "region", "area", "jurisdiction", "bounds"],
    "devolution": ["devolution", "decentralization", "delegation", "transfer of power", "local governance", "subsidiarity"],
    "languages": ["languages", "tongues", "dialects", "official languages", "linguistic diversity"],
    "religion": ["religion", "faith", "belief systems", "spiritual practice", "secularism", "church-state separation"],
    "symbol": ["symbol", "emblem", "insignia", "representation", "national icon"],
    "day": ["day", "holiday", "observance", "public holiday", "commemoration", "remembrance"],
    "value": ["value", "principle", "ethic", "core value", "standard", "national ideal"],
    "governance": ["governance", "government", "administration", "management", "public service", "political structure"],
    "culture": ["culture", "heritage", "tradition", "customs", "societal norms", "arts"],
    "entitlement": ["entitlement", "right", "eligibility", "entitlement rights", "benefits", "privileges"],
    "retention": ["retention", "maintenance", "keeping", "preservation", "continuation"],
    "birth": ["birth", "nativity", "origin", "ancestry", "inborn citizenship"],
    "registration": ["registration", "enlistment", "enrollment", "citizenship application", "naturalization"],
    "dual": ["dual", "multiple", "dual nationality", "two-fold citizenship"],
    "revocation": ["revocation", "cancellation", "annulment", "rescission", "forfeiture", "withdrawal"],
    "legislation": ["legislation", "laws", "legal framework", "statutes", "enactment"],
    "fundamental": ["fundamental", "fundamental freedoms", "essential"],
    "application": ["application", "exercise", "freedom enforcement"],
    "implementation": ["implementation", "right support", "execution"],
    "authority": ["authority", "judicial power", "court jurisdiction", "judicial review"],
    "limitation": ["limitation", "freedom restriction", "limit entitlement"],
    "limit": ["limit","absolute right", "inalienable freedom", "immutable right"],
    "life": ["life", "existence", "entitlement to life", "survive"],
    "equality": ["equality", "equal treatment", "justice"],
    "dignity": ["dignity", "intrinsic respect", "inherent value", "personal honor"],
    "security": ["security", "liberty", "personal safety"],
    "slavery": ["slavery", "bondage", "force servitude", "involuntary labor", "coerce work"],
    "privacy": ["privacy", "confidential", "personal space"],
    "conscience": ["conscience", "liberty of thought", "religion freedom", "belief"],
    "expression": ["expression", "communication", "speak"],
    "medium": ["medium", "press autonomy", "journalist freedom", "right to information"],
    "information": ["information", "transparent", "datum"],
    "association": ["association", "union", "group formation", "association right"],
    "assembly": ["assembly", "dissent", "peace assembly", "demonstrate", "petition right"],
    "political": ["political", "electoral right", "political engagement", "vote right"],
    "movement": ["movement", "migration", "resident freedom", "movement liberty"],
    "Property": ["Property", "property safeguard", "assets security"],
    "work": ["work", "benefit", "employ"],
    "environment": ["environment", "conserve", "right to a healthy environment"],
    "economic": ["economic", "standard of life", "social welfare"],
    "language": ["language", "cultural right", "linguistic freedom", "cultural expression"],
    "family": ["family", "society foundation", "parent right"],
    "consumer": ["consumer", "client entitlement", "buyer safeguard"],
    "administrative": ["administrative", "fair administration rule", "legal administration measure"],
    "justice": ["justice", "legal access", "justice available"],
    "arrest": ["arrest", "capture", "seizure", "apprehension"],
    "fair hearing": ["fair hearing", "prosecute", "equity hearing"],
    "custody": ["custody", "confinement", "imprisonment", "prison"],
    "interpret": ["interpret", "explain", "clarify", "overview"],
    "infant": ["infant", "child", "kid", "toddler"],
    "disable": ["disable", "handicap", "impairment", "challenge"],
    "youth": ["youth", "young adult", "adolescent"],
    "minority": ["minority", "diversity right", "affirmative right"],
    "old": ["old", "elder", "vintage"],
    "emergency": ["emergency", "danger", "crisis", "disaster"],
    "national": ["national", "human right", "equaity Body", "right authority"],
    "principle": ["principle", "land management principle", "land policy guide"],
    "classification": ["classification", "land category", "types of land"],
    "public": ["public", "government land", "state property", "national land", "public landhold"],
    "community": ["community", "ethnic land", "cultural landhold", "community land"],
    "private": ["private", "individual landhold", "personal property", "freehold land", "private land"],
    "landhold" : ["landhold", "foreign lease", "alien land", "non-citizen land"],
    "regulation" : ["regulation", "land use policy", "property regulation", "land oversight"],
    "commission" : ["commission", "land authority", "public land commission", "land policy agency"],
    "land" : ["land", "land legislation", "property law", "land-use regulation", "land act"],
    "obligation" : ["obligation", "biodiversity duty", "conservation mandate"],
    "environmental" : ["environmental", "ecological redress", "sustainability enforcement"],
    "resource" : ["resource", "resource use agreement", "environmental concession"],
    "legislation" : ["legislation", "environmental law", "green policy enactment"],
    "responsibility": ["responsibility", "accountable", "commitment", "charge"],
    "oath": ["oath", "swear", "state office affirmation", "vow", "declare"],
    "conduct": ["conduct", "behavior", "comportment", "demeanor"],
    "finance": ["finance", "finance integrity", "wealth", "invest", "finance management"],
    "restrict": ["restrict", "constrain", "regulate", "prohibit", "curtail", "control"],
    "citizen": ["citizen", "inhabitant", "local resident"],
    "establish": ["establish", "corrupt", "bribe", "fraud"],
    "leader": ["leader", "governing, authority"],
    "judicial authority": ["judicial authority","legal authority","court jurisdiction"],
    "judicial independence": ["judicial independence","judicial autonomy","judicial impartiality"],
    "judicial offices": ["judicial office","legal position","judicial role"],
    "court systems": ["court system","judicial system","legal system"],
    "supreme court": ["supreme court","highest court","apex court"],
    "appeal court": ["appeal court","appellate court","review court"],
    "high court": ["high court","superior court"],
    "judicial appointments": ["judicial appointment","judge appointment","judicial selection"],
    "office tenure": ["office tenure","office term","service duration"],
    "office removal": ["office removal","office dismissal","office termination"],
    "subordinate courts": ["subordinate court","local court","inferior court"],
    "jsc establishment": ["judicial service commission establishment","judicial service commission creation","judicial service commission formation"],
    "jsc functions": ["judicial service commission function","judicial service commission role","judicial service commission responsibility"],
    "judiciary fund": ["judiciary fund","judicial fund","legal fund"],
    "object devolution": ["object devolution", "power decentralization", "government devolution"],
    "devolve government": ["devolve government", "decentralization principle"],
    "county government": ["county government", "local government", "regional administration"],
    "membership": ["membership", "composition", "structure"],
    "speaker": ["speaker", "assembly speaker", "preside officer"],
    "county executive committee": ["county executive committee", "regional executive body", "local executive board"],
    "county election": ["county election", "local election", "regional election"],
    "county removal": ["county removal", "county office dismissal", "regional removal"],
    "vacancy": ["vacancy", "position vacancy", "office vacancy"],
    "functions of county executive committees": ["functions of county executive committees", "duties of county executive commitees", "county executive commitees roles"],
    "city": ["city", "urban center", "municipality"],
    "legislative authority": ["legislative authority", "law make power", "regulatory authority"],
    "function and power of national and county government": ["function and power of national and county government", "government duty", "government responsibility"],
    "transfer": ["transfer", "delegation", "reassignment"],
    "county boundary": ["county boundary", "county border", "regional limit"],
    "cooperation": ["cooperation", "collaboration", "coordination"],
    "support": ["support", "assistance", "aid"],
    "conflict of law": ["conflict of law", "legal discrepancy", "jurisdiction conflict"],
    "suspension": ["suspension", "interruption", "temporary halt"],
    "qualification": ["qualification", "eligibility", "requirement"],
    "vacation": ["vacation", "office leave", "office absence"],
    "summon witness": ["summon witness", "convene witness", "request witness attendance"],
    "public participation county assembly power privilege immunity": ["public participation county assembly power privilege immunity", "county assembly right", "local governance privilege"],
    "gender balance": ["gender balance", "gender equity", "equal representation"],
    "county transition": ["county transition", "regional handover", "local government transition"],
    "publication": ["publication", "official release", "public announcement"],
    "legislation": ["legislation", "regulation", "law"],
    "principle public finance": ["principle public finance", "public finance guideline", "fundamental financial rule"],
    "equitable sharing of national revenue": ["equitable share of national revenue", "fair distribution of revenue"],
    "equitable share law": ["equitable share law", "financial law", "equitable distribution law"],
    "equalisation fund": ["equalisation fund", "equalization", "equal opportunity fund"],
    "consultation": ["consultation", "discussion", "advisement"],
    "consolidated fund": ["consolidated fund", "main fund", "central fund"],
    "revenue funds": ["revenue funds", "revenue", "income fund"],
    "contingencies fund": ["contingency fund", "reserve fund", "emergency fund"],
    "tax impose": ["tax impose", "tax authority", "tax power"],
    "national government borrow": ["national government borrow", "national debt", "federal borrow"],
    "loan guarantee": ["loan guarantee", "credit guarantee", "loan assurance"],
    "public debt": ["public debt", "national debt", "government debt"],
    "commission on revenue allocation": ["commission on revenue allocation", "allocation commission", "revenue allocation committee"],
    "division of revenue": ["division of revenue", "revenue distribution", "allocation of revenue"],
    "budget content": ["budget content", "budget structure", "budget framework"],
    "expenditure": ["expenditure", "cost", "expense"],
    "supplementary appropriation": ["supplementary appropriation", "additional appropriation", "extra allocation"],
    "financial control": ["financial control", "financial oversight", "budget control"],
    "procurement": ["procurement", "acquisition", "purchase"],
    "budget contoller": ["budget contoller", "cost controller"],
    "auditor-general": ["auditor general", "chief auditor", "head auditor"],
    "salary": ["salary","remuneration", "pay"],
    "central bank of kenya": ["central bank of kenya", "central bank", "central financial authority"]

}

# Define qa mapping explicitly too
qa_mapping = {
    "supremacy": "supremacy",
    "sovereignty": "sovereignty",
    "defence": "defence",
    "declaration": "declaration",
    "territory": "territory", 
    "devolution": "devolution",
    "language": "languages",
    "religion": "religion",
    "symbol": "symbol",
    "day": "day",
    "value": "value",
    "culture": "culture",
    "entitlement": "entitlement",
    "retention": "retention",  
    "birth": "birth",
    "registration": "registration",
    "dual": "dual",
    "revocation": "revocation",
    "legislation": "legislation",
    "fundamental": "fundamental",
    "application": "application",
    "implementation": "implementation",
    "authority": "authority",
    "limitation": "limitation",
    "limit": "limit",
    "life": "life",
    "equality": "equality",
    "dignity": "dignity",
    "security": "security",
    "slavery": "slavery",
    "privacy": "privacy",
    "conscience": "conscience",
    "expression": "expression",
    "medium": "medium",
    "information": "information",
    "association": "association",
    "assembly": "assembly",
    "political": "political",
    "movement": "movement",
    "property": "property",
    "work": "work",
    "environment": "environment",
    "economic": "economic",
    "language": "language",
    "family": "family",
    "consumer": "consumer",
    "administrative": "administrative",
    "justice": "justice",
    "arrest": "arrest",
    "fair hearing": "fair hearing",
    "custody": "custody",
    "interpret": "interpret",
    "infant": "infant",
    "disable": "disable",
    "youth": "youth",
    "minority": "minority",
    "old": "old",
    "emergency": "emergency",
    "national": "national",
    "principle": "principle",
    "classification": "classification",
    "public": "public",
    "community": "community",
    "private": "private",
    "landhold": "landhold",
    "regulation": "regulation",
    "commission": "commission",
    "land": "land",
    "obligation": "obligation",
    "environmental": "environmental",
    "resource": "resource",
    "legislation": "legislation",
    "responsibility": "responsibility",
    "oath": "oath",
    "conduct": "conduct",
    "finance": "finance",
    "restrict": "restrict",
    "citizen": "citizen",
    "establish": "establish",
    "leader": "leader",
    "judicial authority": "judicial authority",
    "judicial independence" :"judiciary independence",
    "judicial offices": "judicial offices",
    "court systems": "court systems",
    "supreme court": "supreme court",
    "appeal court": "appeal court",
    "high court": "high court",
    "judicial appointments": "judicial appointments",
    "office tenure": "office tenure",
    "office removal": "office removal",
    "subordinate courts": "subordinate courts",
    "jsc establishment": "jsc establishment",
    "jsc functions": "jsc functions",
    "judiciary fund": "judiciary fund",
    "object devolution": "object devolution",
    "devolve government": "devolve government",
    "county government": "county government",
    "membership": "membership",
    "speaker": "speaker",
    "county executive committee": "county executive committee",
    "county election": "county election",
    "county removal": "county removal",
    "vacancy": "vacancy",
    "functions of county executive committees": "executive function",
    "city": "city",
    "legislative authority": "legislative authority",
    "function and power of national and county government": "function and power of national and county government",
    "transfer": "transfer",
    "county boundary": "county boundary",
    "cooperation": "cooperation",
    "support": "support",
    "conflict of law": "law conflict",
    "suspension": "suspension county government",
    "qualification": "qualification",
    "vacation": "vacation",
    "summon witness": "summon witness",
    "public participation county assembly power privilege immunity": "public participation county assembly power privilege immunity",
    "gender balance": "gender balance",
    "county transition": "transition",
    "publication": "publication",
    "legislation": "legislation",
    "principle public finance": "principle public finance",
    "equitable sharing of national revenue": "equitable sharing of national revenue",
    "equitable share law": "equitable share",
    "equalisation fund": "equalisation fund",
    "consultation": "consultation",
    "consolidated fund": "consolidate",
    "revenue funds": "revenue",
    "contingencies fund": "contingency",
    "tax impose": "tax",
    "national government borrow": "borrow",
    "loan guarantee": "loan",
    "public debt": "debt",
    "commission on revenue allocation": "commission",
    "division of revenue": "division",
    "budget content": "budget",
    "expenditure": "expenditure",
    "supplementary appropriation": "supplementary",
    "financial control": "control",
    "procurement": "procurement",
    "budget controller": "budget controller",
    "auditor-general": "auditor",
    "salary": "salary",
    "central bank of kenya": "bank"

    
}

combined_sections = {**chapter_1_sections, **chapter_2_sections, **chapter_3_sections, **chapter_4_sections, **chapter_5_sections, **chapter_6_sections, **chapter_10_sections, **chapter_11_sections, **chapter_12_sections }
# Print the keys of the dictionary
len(combined_sections.keys())

151

# CHAPTER 13

In [51]:
chapter_13 = extract_specific_pages(pdf_path, 138,143)
chapter_13_trimmed = chapter_13.split("CHAPTER FOURTEEN")[0].strip()


In [52]:
## load spacy
nlp = spacy.load("en_core_web_sm")

## preprocess the user query using spacy

def preprocess_query(query):
  ## Parse the query with spacy
  doc = nlp(query)
  ## Normalize the query: lowercase,lemmatize and remove stopwords
  tokens = [token.lemma_.lower()for token in doc if not token.is_stop and not token.is_punct]
  return " ".join(tokens)

## Example
user_query = "What are Values and principles of public service?"
processed_query= preprocess_query(user_query)
print(processed_query)


value principle public service


In [53]:
def split_chapter(chapter_text):
  #split at key headings and split extra white spaces

  sections = {
      "principle":[],
      "public service commission":[],
      "functions":[],
      "staffing":[],
      "protection":[],
      "teacher service commission":[]
  }

  ## Split by new lines to process line by line
  lines = chapter_text.splitlines()
  current_section = None

  for line in lines:
    line = line.strip()
    # Use line instead of stripped_line
    if line.startswith("Values and principles of public service"):
      current_section = "principle"
    elif line.startswith("The Public Service Commission"):
        current_section = "public service commission"
    elif line.startswith("Functions and powers of the Public Service Commission"):
        current_section = "functions"
    elif line.startswith("Staffing of county governments"):
        current_section = "staffing"
    elif line.startswith("Protection of public officers"):
        current_section = "protection"
    elif line.startswith("Teachers Service Commission"):
            current_section = "teacher service commission"

    # Append line to the current section if it's set
    if current_section:
        sections[current_section].append(line) # Use line instead of stripped_line
  ## Join each section into single string
  for key in sections:
    sections[key] = "\n".join(sections[key])
  return sections

chapter_13_sections = split_chapter(chapter_13_trimmed)

## Question answering mechanism

In [54]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Preprocess the user query using spaCy
def preprocess_query(query):
    # Parse the query with spaCy
    doc = nlp(query)
    # Normalize the query: lowercase, lemmatize, and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Define the sections variable with Chapter 1
sections = chapter_13_sections

# Define the QA mapping based on key phrases and corresponding sections
qa_mapping = {
    "principle": "Values and principles of public service",
    "public service commission": "The Public Service Commission",
    "functions": "Functions and powers of the Public Service Commission",
    "staffing" : "Staffing of county governments",
    "protection": "Protection of public officers",
    "teacher service commission": "Teachers Service Commision"

}

# Update the Q&A system to use preprocessed queries
def answer_question_nlp(query, sections, qa_mapping):
    # Preprocess the user query
    processed_query = preprocess_query(query)
    
    # Search for a key in qa_mapping that matches the preprocessed query
    for key in qa_mapping:
        if key in processed_query:
            # Return the relevant section in the text
            return sections[key]
            
    return "Sorry, I couldn't find an answer to your question."

# Example usage
user_query = "What about teacher service commission?"
answer = answer_question_nlp(user_query, sections, qa_mapping)
print(answer)


Teachers Service Commission.
237. (1) There is established the Teachers Service Commission.
(2) The functions of the Commission are—
(a) to register trained teachers;
(b) to recruit and employ registered teachers;
Constitution of Kenya, 2010 143
(c) to assign teachers employed by the Commission for service in
any public school or institution;
(d) to promote and transfer teachers;
(e) to exercise disciplinary control over teachers; and
(f) to terminate the employment of teachers.
(3) The Commission shall—
(a) review the standards of education and training of persons
entering the teaching service;
(b) review the demand for and the supply of teachers; and
(c) advise the national government on matters relating to the
teaching profession.


### Chapter 13 NLU

# CHAPTER 14

In [55]:
chapter_14 = extract_specific_pages(pdf_path, 142,150)
# Trim the chapter_14 text to start from "CHAPTER FOURTEEN"
chapter_14_trimmed = chapter_14[chapter_14.find("CHAPTER FOURTEEN"):].strip()

## Further trim to remove Chapter 15 if it's present on the same page
if "CHAPTER FIFTEEN" in chapter_14_trimmed:
  chapter_14_trimmed = chapter_14_trimmed.split("CHAPTER FIFTEEN")[0].strip()

In [56]:
## load spacy
nlp = spacy.load("en_core_web_sm")

## preprocess the user query using spacy

def preprocess_query(query):
  ## Parse the query with spacy
  doc = nlp(query)
  ## Normalize the query: lowercase,lemmatize and remove stopwords
  tokens = [token.lemma_.lower()for token in doc if not token.is_stop and not token.is_punct]
  return " ".join(tokens)

## Example
user_query = "What are the principles of national security?"
preprocess_query= preprocess_query(user_query)
print(preprocess_query)


principle national security


In [57]:
def split_chapter(chapter_text):
    # Define sections with lowercase keys
    sections = {
        "national security": [],
        "national security organs": [],
        "national security council": [],
        "defence forces": [],
        "national intelligence service": [],
        "national police service": [],
        "functions national police service": [],
        "command national police service": [],
        "national police service commission": [],
        "police service": []
    }

    # Split by new lines to process line by line
    lines = chapter_text.splitlines()


    current_section = None

    for line in lines:
        stripped_line = line.strip()

        # Match line starts with capitalized section headers
        if stripped_line.startswith("Principles of national security"):
            current_section = "national security"
        elif stripped_line.startswith("National security organs"):
            current_section = "national security organs"
        elif stripped_line.startswith("Establishment of the National Security Council"):
            current_section = "national security council"
        elif stripped_line.startswith("Establishment of Defence Forces and Defence Council"):
            current_section = "defence forces"
        elif stripped_line.startswith("Establishment of National Intelligence Service"):
            current_section = "national intelligence service"
        elif stripped_line.startswith("Establishment of the National Police Service"):
            current_section = "national police service"
        elif stripped_line.startswith("Objects and Functions of the National Police Service"):
            current_section = "functions national police service"
        elif stripped_line.startswith("Command of the National Police Service"):
            current_section = "command national police service"
        elif stripped_line.startswith("National Police Service Commission"):
            current_section = "national police service commission"
        elif stripped_line.startswith("Other Police Services"):
            current_section = "police service"

        # Append line to the current section if it's set
        if current_section:
            sections[current_section].append(line)

    # Join each section into a single string
    for key in sections:
        sections[key] = "\n".join(sections[key])

    return sections

# Apply the function to the chapter 14
chapter_14_sections = split_chapter(chapter_14_trimmed)


In [58]:
chapter_14_sections

{'national security': 'Principles of national security.\n238. (1) National security is the protection against internal and\nexternal threats to Kenya’s territorial integrity and sovereignty, its\npeople, their rights, freedoms, property, peace, stability and prosperity,\nand other national interests.\n(2) The national security of Kenya shall be promoted and\nguaranteed in accordance with the following principles—\n(a) national security is subject to the authority of this Constitution\nand Parliament;\n(b) national security shall be pursued in compliance with the law\nand with the utmost respect for the rule of law, democracy,\nhuman rights and fundamental freedoms;\n(c) in performing their functions and exercising their powers,\nnational security organs shall respect the diverse culture of\nthe communities within Kenya; and\n(d) recruitment by the national security organs shall reflect the\ndiversity of the Kenyan people in equitable proportions.',
 'national security organs': 'Nationa

## Question answering mechanism

In [59]:
qa_mapping = {
    "national security": "national security",
    "national security organs": "national security organs",
    "national security council": "national security council",
    "defence forces and defence council": "defence forces",
    "national intelligence service": "national intelligence service",
    "national police service": "national police service",
    "functions national police service": "functions national police service",
    "command national police service": "command national police service",
    "national police service commission": "national police service commission",
    "other police services": "other police services"
}

# Preprocess the user query using spaCy
def preprocess_query(query):
    # Parse the query with spaCy
    doc = nlp(query)
    # Normalize the query: lowercase, lemmatize, and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Update the Q&A system to use preprocessed queries
def answer_question_nlp(query, sections, qa_mapping):
    # Preprocess the user query
    processed_query = preprocess_query(query)

    # Debug
    print(f"Processed query: {processed_query}")

    # Search for a key in qa_mapping that matches the preprocessed query
    for key in qa_mapping:
        if key in processed_query:
            # Debug line
            print(f"key: {key}")
            # Return the relevant section in the text
            return sections[key]

    return "Sorry, I couldn't find an answer to your question."

##Example
user_query = "What are the principles of national security?"
answer = answer_question_nlp(user_query,chapter_14_sections, qa_mapping) # Pass the user_query
answer

Processed query: principle national security
key: national security


'Principles of national security.\n238. (1) National security is the protection against internal and\nexternal threats to Kenya’s territorial integrity and sovereignty, its\npeople, their rights, freedoms, property, peace, stability and prosperity,\nand other national interests.\n(2) The national security of Kenya shall be promoted and\nguaranteed in accordance with the following principles—\n(a) national security is subject to the authority of this Constitution\nand Parliament;\n(b) national security shall be pursued in compliance with the law\nand with the utmost respect for the rule of law, democracy,\nhuman rights and fundamental freedoms;\n(c) in performing their functions and exercising their powers,\nnational security organs shall respect the diverse culture of\nthe communities within Kenya; and\n(d) recruitment by the national security organs shall reflect the\ndiversity of the Kenyan people in equitable proportions.'

# CHAPTER 15

In [60]:
chapter_15 = extract_specific_pages(pdf_path, 148,155)

In [61]:
# Trim the chapter_15 text to start from "CHAPTER FIFTEEN"
chapter_15_trimmed = chapter_15[chapter_15.find("CHAPTER FIFTEEN"):].strip()

# Further trim to remove Chapter 16 if it's present on the same page
if "CHAPTER SIXTEEN" in chapter_15_trimmed:
  chapter_15_trimmed = chapter_15_trimmed.split("CHAPTER SIXTEEN")[0].strip()

chapter_15_trimmed

'CHAPTER FIFTEEN—COMMISSIONS AND INDEPENDENT\nOFFICES\nApplication of Chapter.\n248. (1) This Chapter applies to the commissions specified in\nclause (2) and the independent offices specified in clause (3), except\nto the extent that this Constitution provides otherwise.\n(2) The commissions are—\n150 Constitution of Kenya, 2010\n(a) the Kenya National Human Rights and Equality Commission;\n(b) the National Land Commission;\n(c) the Independent Electoral and Boundaries Commission;\n(d) the Parliamentary Service Commission;\n(e) the Judicial Service Commission;\n(f) the Commission on Revenue Allocation;\n(g) the Public Service Commission;\n(h) the Salaries and Remuneration Commission;\n(i) the Teachers Service Commission; and\n(j) the National Police Service Commission.\n(3) The independent offices are—\n(a) the Auditor-General; and\n(b) the Controller of Budget.\nObjects, authority and funding of commissions and independent\noffices.\n249. (1) The objects of the commissions and the ind

In [62]:
## preprocess the user query using spacy

def preprocess_query(query):
  ## Parse the query with spacy
  doc = nlp(query)
  ## Normalize the query: lowercase,lemmatize and remove stopwords
  tokens = [token.lemma_.lower()for token in doc if not token.is_stop and not token.is_punct]
  return " ".join(tokens)

## Example
user_query = "What are the commissions and independent offices?"
preprocess_query= preprocess_query(user_query)
print(preprocess_query)

commission independent office


In [63]:
def split_chapter_15(chapter_text):
  #split at key headings and split extra white spaces
  # Initialize sections as a dictionary with list values
  sections = {
        "application of chapter": [],
        "commission independent office": [],
        "term of office": [],
        "removal from office": [],
        "general function power": [],
        "incorporation commission independent office": [],
        "reporting commission independent office": []
    }

  # Split by new lines to process line by line
  lines = chapter_text.splitlines()
  current_section = None

  for line in lines:
    stripped_line = line.strip()

    if line.startswith("Application of Chapter"):
          current_section = "application of chapter"
    elif line.startswith("Objects,authority and funding of commissions and independent offices"):
          current_section = "commission independent office"
    elif line.startswith("Composition, appointment and terms of office"):
          current_section = "term of office"
    elif line.startswith("Removal from office"):
          current_section = "removal from office"
    elif line.startswith("General functions and powers"):
          current_section = "general function power"
    elif line.startswith("Incorporation of commissions and independent offices"):
          current_section = "incorporation commission independent office"
    elif line.startswith("Reporting by commissions and independent offices"):
          current_section = "reporting commission independent office"
          ## Append line to the current section if it's set

    if current_section:
            sections[current_section].append(stripped_line)

  # Join each section into single string
  for key in sections:
        sections[key] = "\n".join(sections[key])
  return sections

chapter_15_sections = split_chapter_15(chapter_15_trimmed)

In [64]:
# Define sections with lemmatized keys
sections = chapter_15_sections

# Ensure qa_mapping uses lemmatized keys
qa_mapping = {
    "application chapter": "application chapter",
    "commission independent office": "commission independent office",  # Use the actual key from 'sections'
    "term office": "term office",
    "removal office": "removal office",
    "general function power": "general function power",
    "incorporation commission independent office": "incorporation commission independent office",
    "reporting commissions independent office": "reporting commission independent office"
}


def preprocess_query(query):
    ## Parse the query with spacy
    doc = nlp(query)
    ## Normalize the query: lowercase,lemmatize and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)


# Update the Q&A system to use preprocessed queries
def answer_question_nlp(query, sections, qa_mapping):
    # Preprocess the user query
    processed_query = preprocess_query(query)

    # Debug
    print(f"Processed query: {processed_query}")

    # Search for a key in qa_mapping that matches the preprocessed query
    for key in qa_mapping:
        if key in processed_query:
            # Debug line
            print(f"key: {key}")
            # Return the relevant section in the text
            return sections[key]  # Access using the original key
    return "Sorry, I couldn't find an answer to your question."

# Example usage
user_query = "What about the commission independent office?"
answer = answer_question_nlp(user_query, sections, qa_mapping)
print(answer)

Processed query: commission independent office
key: commission independent office



# Cumulative sections, qa mapping and synonyms

In [65]:
# Define the synonyms explicitly so that we can easily reference them later
synonyms = {
    "supremacy": ["supremacy", "authority", "ultimate power"],
    "sovereignty": ["sovereignty", "power of the people", "authority of the people", "self rule", "autonomy"],
    "defence": ["defense", "protection", "preservation"],
    "declaration": ["declaration", "proclamation", "statement", "announcement", "affirmation"],
    "territory": ["territory", "land", "region", "area", "jurisdiction", "bounds"],
    "devolution": ["devolution", "decentralization", "delegation", "transfer of power", "local governance", "subsidiarity"],
    "languages": ["languages", "tongues", "dialects", "official languages", "linguistic diversity"],
    "religion": ["religion", "faith", "belief systems", "spiritual practice", "secularism", "church-state separation"],
    "symbol": ["symbol", "emblem", "insignia", "representation", "national icon"],
    "day": ["day", "holiday", "observance", "public holiday", "commemoration", "remembrance"],
    "value": ["value", "principle", "ethic", "core value", "standard", "national ideal"],
    "governance": ["governance", "government", "administration", "management", "public service", "political structure"],
    "culture": ["culture", "heritage", "tradition", "customs", "societal norms", "arts"],
    "entitlement": ["entitlement", "right", "eligibility", "entitlement rights", "benefits", "privileges"],
    "retention": ["retention", "maintenance", "keeping", "preservation", "continuation"],
    "birth": ["birth", "nativity", "origin", "ancestry", "inborn citizenship"],
    "registration": ["registration", "enlistment", "enrollment", "citizenship application", "naturalization"],
    "dual": ["dual", "multiple", "dual nationality", "two-fold citizenship"],
    "revocation": ["revocation", "cancellation", "annulment", "rescission", "forfeiture", "withdrawal"],
    "legislation": ["legislation", "laws", "legal framework", "statutes", "enactment"],
    "fundamental": ["fundamental", "fundamental freedoms", "essential"],
    "application": ["application", "exercise", "freedom enforcement"],
    "implementation": ["implementation", "right support", "execution"],
    "authority": ["authority", "judicial power", "court jurisdiction", "judicial review"],
    "limitation": ["limitation", "freedom restriction", "limit entitlement"],
    "limit": ["limit","absolute right", "inalienable freedom", "immutable right"],
    "life": ["life", "existence", "entitlement to life", "survive"],
    "equality": ["equality", "equal treatment", "justice"],
    "dignity": ["dignity", "intrinsic respect", "inherent value", "personal honor"],
    "security": ["security", "liberty", "personal safety"],
    "slavery": ["slavery", "bondage", "force servitude", "involuntary labor", "coerce work"],
    "privacy": ["privacy", "confidential", "personal space"],
    "conscience": ["conscience", "liberty of thought", "religion freedom", "belief"],
    "expression": ["expression", "communication", "speak"],
    "medium": ["medium", "press autonomy", "journalist freedom", "right to information"],
    "information": ["information", "transparent", "datum"],
    "association": ["association", "union", "group formation", "association right"],
    "assembly": ["assembly", "dissent", "peace assembly", "demonstrate", "petition right"],
    "political": ["political", "electoral right", "political engagement", "vote right"],
    "movement": ["movement", "migration", "resident freedom", "movement liberty"],
    "Property": ["Property", "property safeguard", "assets security"],
    "work": ["work", "benefit", "employ"],
    "environment": ["environment", "conserve", "right to a healthy environment"],
    "economic": ["economic", "standard of life", "social welfare"],
    "language": ["language", "cultural right", "linguistic freedom", "cultural expression"],
    "family": ["family", "society foundation", "parent right"],
    "consumer": ["consumer", "client entitlement", "buyer safeguard"],
    "administrative": ["administrative", "fair administration rule", "legal administration measure"],
    "justice": ["justice", "legal access", "justice available"],
    "arrest": ["arrest", "capture", "seizure", "apprehension"],
    "fair hearing": ["fair hearing", "prosecute", "equity hearing"],
    "custody": ["custody", "confinement", "imprisonment", "prison"],
    "interpret": ["interpret", "explain", "clarify", "overview"],
    "infant": ["infant", "child", "kid", "toddler"],
    "disable": ["disable", "handicap", "impairment", "challenge"],
    "youth": ["youth", "young adult", "adolescent"],
    "minority": ["minority", "diversity right", "affirmative right"],
    "old": ["old", "elder", "vintage"],
    "emergency": ["emergency", "danger", "crisis", "disaster"],
    "national": ["national", "human right", "equaity Body", "right authority"],
    "principle": ["principle", "land management principle", "land policy guide"],
    "classification": ["classification", "land category", "types of land"],
    "public": ["public", "government land", "state property", "national land", "public landhold"],
    "community": ["community", "ethnic land", "cultural landhold", "community land"],
    "private": ["private", "individual landhold", "personal property", "freehold land", "private land"],
    "landhold" : ["landhold", "foreign lease", "alien land", "non-citizen land"],
    "regulation" : ["regulation", "land use policy", "property regulation", "land oversight"],
    "commission" : ["commission", "land authority", "public land commission", "land policy agency"],
    "land" : ["land", "land legislation", "property law", "land-use regulation", "land act"],
    "obligation" : ["obligation", "biodiversity duty", "conservation mandate"],
    "environmental" : ["environmental", "ecological redress", "sustainability enforcement"],
    "resource" : ["resource", "resource use agreement", "environmental concession"],
    "legislation" : ["legislation", "environmental law", "green policy enactment"],
    "responsibility": ["responsibility", "accountable", "commitment", "charge"],
    "oath": ["oath", "swear", "state office affirmation", "vow", "declare"],
    "conduct": ["conduct", "behavior", "comportment", "demeanor"],
    "finance": ["finance", "finance integrity", "wealth", "invest", "finance management"],
    "restrict": ["restrict", "constrain", "regulate", "prohibit", "curtail", "control"],
    "citizen": ["citizen", "inhabitant", "local resident"],
    "establish": ["establish", "corrupt", "bribe", "fraud"],
    "leader": ["leader", "governing, authority"],
    "judicial authority": ["judicial authority","legal authority","court jurisdiction"],
    "judicial independence": ["judicial independence","judicial autonomy","judicial impartiality"],
    "judicial offices": ["judicial office","legal position","judicial role"],
    "court systems": ["court system","judicial system","legal system"],
    "supreme court": ["supreme court","highest court","apex court"],
    "appeal court": ["appeal court","appellate court","review court"],
    "high court": ["high court","superior court"],
    "judicial appointments": ["judicial appointment","judge appointment","judicial selection"],
    "office tenure": ["office tenure","office term","service duration"],
    "office removal": ["office removal","office dismissal","office termination"],
    "subordinate courts": ["subordinate court","local court","inferior court"],
    "jsc establishment": ["judicial service commission establishment","judicial service commission creation","judicial service commission formation"],
    "jsc functions": ["judicial service commission function","judicial service commission role","judicial service commission responsibility"],
    "judiciary fund": ["judiciary fund","judicial fund","legal fund"],
    "object devolution": ["object devolution", "power decentralization", "government devolution"],
    "devolve government": ["devolve government", "decentralization principle"],
    "county government": ["county government", "local government", "regional administration"],
    "membership": ["membership", "composition", "structure"],
    "speaker": ["speaker", "assembly speaker", "preside officer"],
    "county executive committee": ["county executive committee", "regional executive body", "local executive board"],
    "county election": ["county election", "local election", "regional election"],
    "county removal": ["county removal", "county office dismissal", "regional removal"],
    "vacancy": ["vacancy", "position vacancy", "office vacancy"],
    "functions of county executive committees": ["functions of county executive committees", "duties of county executive commitees", "county executive commitees roles"],
    "city": ["city", "urban center", "municipality"],
    "legislative authority": ["legislative authority", "law make power", "regulatory authority"],
    "function and power of national and county government": ["function and power of national and county government", "government duty", "government responsibility"],
    "transfer": ["transfer", "delegation", "reassignment"],
    "county boundary": ["county boundary", "county border", "regional limit"],
    "cooperation": ["cooperation", "collaboration", "coordination"],
    "support": ["support", "assistance", "aid"],
    "conflict of law": ["conflict of law", "legal discrepancy", "jurisdiction conflict"],
    "suspension": ["suspension", "interruption", "temporary halt"],
    "qualification": ["qualification", "eligibility", "requirement"],
    "vacation": ["vacation", "office leave", "office absence"],
    "summon witness": ["summon witness", "convene witness", "request witness attendance"],
    "public participation county assembly power privilege immunity": ["public participation county assembly power privilege immunity", "county assembly right", "local governance privilege"],
    "gender balance": ["gender balance", "gender equity", "equal representation"],
    "county transition": ["county transition", "regional handover", "local government transition"],
    "publication": ["publication", "official release", "public announcement"],
    "legislation": ["legislation", "regulation", "law"],
    "principle public finance": ["principle public finance", "public finance guideline", "fundamental financial rule"],
    "equitable sharing of national revenue": ["equitable share of national revenue", "fair distribution of revenue"],
    "equitable share law": ["equitable share law", "financial law", "equitable distribution law"],
    "equalisation fund": ["equalisation fund", "equalization", "equal opportunity fund"],
    "consultation": ["consultation", "discussion", "advisement"],
    "consolidated fund": ["consolidated fund", "main fund", "central fund"],
    "revenue funds": ["revenue funds", "revenue", "income fund"],
    "contingencies fund": ["contingency fund", "reserve fund", "emergency fund"],
    "tax impose": ["tax impose", "tax authority", "tax power"],
    "national government borrow": ["national government borrow", "national debt", "federal borrow"],
    "loan guarantee": ["loan guarantee", "credit guarantee", "loan assurance"],
    "public debt": ["public debt", "national debt", "government debt"],
    "commission on revenue allocation": ["commission on revenue allocation", "allocation commission", "revenue allocation committee"],
    "division of revenue": ["division of revenue", "revenue distribution", "allocation of revenue"],
    "budget content": ["budget content", "budget structure", "budget framework"],
    "expenditure": ["expenditure", "cost", "expense"],
    "supplementary appropriation": ["supplementary appropriation", "additional appropriation", "extra allocation"],
    "financial control": ["financial control", "financial oversight", "budget control"],
    "procurement": ["procurement", "acquisition", "purchase"],
    "budget contoller": ["budget contoller", "cost controller"],
    "auditor-general": ["auditor general", "chief auditor", "head auditor"],
    "salary": ["salary","remuneration", "pay"],
    "central bank of kenya": ["central bank of kenya", "central bank", "central financial authority"],
    "principle": ["principle", "values and principles of public service", "public service values", "core principles of public service"],
    "public service commission": ["public service commission", "the public service commission", "public service authority", "government service commission"],
    "functions": ["functions", "functions and powers of the public service commission", "roles of the public service commission", "duties of the public service commission"],
    "staffing": ["staffing", "staffing of county governments", "county government staffing", "human resources for county governments"],
    "protection": ["protection", "protection of public officers", "safeguarding public officers", "security for public officials"],
    "teachers service commission": ["teachers service commission", "commission for teachers", "educators service commission"],
    "national security": ["national security", "principles of national security"],
    "national security organs": ["national security organs", "security organs", "national security agencies"],
    "national security council": ["national security council", "establishment of the national security council", "national security council formation", "creation of the national security council"],
    "defence forces": ["defence forces and defence council", "defence forces establishment", "formation of defence forces and council"],
    "national intelligence service": ["national intelligence service", "establishment of national intelligence service", "national intelligence service creation", "formation of national intelligence service"],
    "national police service": ["national police service", "establishment of the national police service", "national police service formation", "creation of the national police service"],
    "functions national police service": ["functions national police service", "objects and functions of the national police service", "roles of the national police service", "duties and functions of the national police"],
    "command national police service": ["command national police service", "command of the national police service", "national police service command", "leadership of the national police service"],
    "national police service commission": ["national police service commission", "police service commission", "commission for the national police service"],
    "other police services": ["other police services", "additional police services", "alternative police services"],
    "application of chapter": ["application of chapter", "chapter application", "provisions of the chapter"],
    "commission independent office": ["commission independent office", "commissions and independent offices", "independent commissions", "commission offices"],
    "term office": ["term office", "term office", "office term", "tenure condition"],
    "removal office": ["removal office", "dismissal from office", "termination of office"],
    "general function power": ["general function power", "overall powers", "general powers and functions"],
    "incorporation commission independent office": ["incorporation commission independent office", "incorporation commissions and independent offices", "establishment of commissions", "setting up independent offices"],
    "reporting commission independent office": ["reporting commissions independent office", "reporting commissions and independent offices", "commissions reporting", "reporting by independent offices"]


}

# Define qa mapping explicitly too
qa_mapping = {
    "supremacy": "supremacy",
    "sovereignty": "sovereignty",
    "defence": "defence",
    "declaration": "declaration",
    "territory": "territory", 
    "devolution": "devolution",
    "language": "languages",
    "religion": "religion",
    "symbol": "symbol",
    "day": "day",
    "value": "value",
    "culture": "culture",
    "entitlement": "entitlement",
    "retention": "retention",  
    "birth": "birth",
    "registration": "registration",
    "dual": "dual",
    "revocation": "revocation",
    "legislation": "legislation",
    "fundamental": "fundamental",
    "application": "application",
    "implementation": "implementation",
    "authority": "authority",
    "limitation": "limitation",
    "limit": "limit",
    "life": "life",
    "equality": "equality",
    "dignity": "dignity",
    "security": "security",
    "slavery": "slavery",
    "privacy": "privacy",
    "conscience": "conscience",
    "expression": "expression",
    "medium": "medium",
    "information": "information",
    "association": "association",
    "assembly": "assembly",
    "political": "political",
    "movement": "movement",
    "property": "property",
    "work": "work",
    "environment": "environment",
    "economic": "economic",
    "language": "language",
    "family": "family",
    "consumer": "consumer",
    "administrative": "administrative",
    "justice": "justice",
    "arrest": "arrest",
    "fair hearing": "fair hearing",
    "custody": "custody",
    "interpret": "interpret",
    "infant": "infant",
    "disable": "disable",
    "youth": "youth",
    "minority": "minority",
    "old": "old",
    "emergency": "emergency",
    "national": "national",
    "principle": "principle",
    "classification": "classification",
    "public": "public",
    "community": "community",
    "private": "private",
    "landhold": "landhold",
    "regulation": "regulation",
    "commission": "commission",
    "land": "land",
    "obligation": "obligation",
    "environmental": "environmental",
    "resource": "resource",
    "legislation": "legislation",
    "responsibility": "responsibility",
    "oath": "oath",
    "conduct": "conduct",
    "finance": "finance",
    "restrict": "restrict",
    "citizen": "citizen",
    "establish": "establish",
    "leader": "leader",
    "judicial authority": "judicial authority",
    "judicial independence" :"judiciary independence",
    "judicial offices": "judicial offices",
    "court systems": "court systems",
    "supreme court": "supreme court",
    "appeal court": "appeal court",
    "high court": "high court",
    "judicial appointments": "judicial appointments",
    "office tenure": "office tenure",
    "office removal": "office removal",
    "subordinate courts": "subordinate courts",
    "jsc establishment": "jsc establishment",
    "jsc functions": "jsc functions",
    "judiciary fund": "judiciary fund",
    "object devolution": "object devolution",
    "devolve government": "devolve government",
    "county government": "county government",
    "membership": "membership",
    "speaker": "speaker",
    "county executive committee": "county executive committee",
    "county election": "county election",
    "county removal": "county removal",
    "vacancy": "vacancy",
    "functions of county executive committees": "executive function",
    "city": "city",
    "legislative authority": "legislative authority",
    "function and power of national and county government": "function and power of national and county government",
    "transfer": "transfer",
    "county boundary": "county boundary",
    "cooperation": "cooperation",
    "support": "support",
    "conflict of law": "law conflict",
    "suspension": "suspension county government",
    "qualification": "qualification",
    "vacation": "vacation",
    "summon witness": "summon witness",
    "public participation county assembly power privilege immunity": "public participation county assembly power privilege immunity",
    "gender balance": "gender balance",
    "county transition": "transition",
    "publication": "publication",
    "legislation": "legislation",
    "principle public finance": "principle public finance",
    "equitable sharing of national revenue": "equitable sharing of national revenue",
    "equitable share law": "equitable share",
    "equalisation fund": "equalisation fund",
    "consultation": "consultation",
    "consolidated fund": "consolidate",
    "revenue funds": "revenue",
    "contingencies fund": "contingency",
    "tax impose": "tax",
    "national government borrow": "borrow",
    "loan guarantee": "loan",
    "public debt": "debt",
    "commission on revenue allocation": "commission",
    "division of revenue": "division",
    "budget content": "budget",
    "expenditure": "expenditure",
    "supplementary appropriation": "supplementary",
    "financial control": "control",
    "procurement": "procurement",
    "budget controller": "budget controller",
    "auditor-general": "auditor",
    "salary": "salary",
    "central bank of kenya": "bank",
    "principle": "principle",
    "public service commission": "public service commission",
    "functions": "functions",
    "staffing": "staffing",
    "protection": "protection",
    "teachers service commission": "teachers service commission",
    "national security": "national security",
    "national security organs": "national security organs",
    "national security council": "national security council",
    "defence forces": "defence forces",
    "national intelligence service": "national intelligence service",
    "national police service": "national police service",
    "functions national police service": "functions national police service",
    "command national police service": "command national police service",
    "national police service commission": "national police service commission",
    "other police services": "other police services",
    "application of chapter": "application of chapter",
    "commission independent office": "commission independent office",
    "term of office": "term of office",
    "removal from office": "removal from office",
    "general function power": "general function power",
    "incorporation commission independent office": "incorporation commission independent office",
    "reporting commission independent office": "reporting commission independent office"


}

combined_sections = {**chapter_1_sections, **chapter_2_sections, **chapter_3_sections, **chapter_4_sections,
                     **chapter_5_sections, **chapter_6_sections, **chapter_10_sections, **chapter_11_sections,
                     **chapter_12_sections, **chapter_13_sections, **chapter_14_sections, **chapter_15_sections}
# Print the keys of the dictionary
len(combined_sections.keys())

173

# CHAPTER 16

In [66]:
chapter_16 = extract_specific_pages(pdf_path, 153,157)
# Trim chapter 16 text to start from chapter sixteen
chapter_16_trimmed = chapter_16[chapter_16.find('CHAPTER SIXTEEN'):].strip()

## Further trim to remove chapter 17 if its present on the same page

if "CHAPTER SEVENTEEN" in chapter_16_trimmed:
  chapter_16_trimmed = chapter_16_trimmed.split("CHAPTER SEVENTEEN")[0].strip()

print(chapter_16_trimmed)


CHAPTER SIXTEEN—AMENDMENT OF THIS CONSTITUTION
Amendment of this Constitution.
255. (1) A proposed amendment to this Constitution shall be
enacted in accordance with Article 256 or 257, and approved in
accordance with clause (2) by a referendum, if the amendment relates
to any of the following matters—
(a) the supremacy of this Constitution;
(b) the territory of Kenya;
(c) the sovereignty of the people;
(d) the national values and principles of governance referred to in
Article 10 (2) (a) to (d);
(e) the Bill of Rights;
(f) the term of office of the President;
(g) the independence of the Judiciary and the commissions and
independent offices to which Chapter Fifteen applies;
(h) the functions of Parliament;
Constitution of Kenya, 2010 155
(i) the objects, principles and structure of devolved government;
or
(j) the provisions of this Chapter.
(2) A proposed amendment shall be approved by a referendum
under clause (1) if—
(a) at least twenty per cent of the registered voters in each of at

In [67]:
## load spacy
nlp = spacy.load("en_core_web_sm")

## preprocess the user query using spacy

def preprocess_query(query):
  ## Parse the query with spacy
  doc = nlp(query)
  ## Normalize the query: lowercase,lemmatize and remove stopwords
  tokens = [token.lemma_.lower()for token in doc if not token.is_stop and not token.is_punct]
  return " ".join(tokens)

## Example
user_query = "What is the amendment of this constitution?"
preprocess_query= preprocess_query(user_query)
print(preprocess_query)


amendment constitution


In [68]:
def split_chapter(chapter_text):
  ## split at key headings and split extra white spaces
  sections = {
      "amendment constitution":[],  # Changed key to match the assignment in the loop
      "parliamentary initiative":[], # Changed key to match the assignment in the loop
      "popular initiative":[] # Changed key to match the assignment in the loop

  }

  ##Split by new lines to process line by line
  lines = chapter_text.split("\n")
  current_section = None

  for line in lines:
    line = line.strip()
    if line.startswith("Amendment"):
      current_section = "amendment constitution" # This line now correctly assigns to an existing key
    elif line.startswith("Parliamentary Initiative"):
      current_section = "Amendment by parliamentary initiative" # This line now correctly assigns to an existing key
    elif line.startswith("Popular Initiative"):
      current_section = "popular initiative" # This line now correctly assigns to an existing key
    elif current_section:
      sections[current_section].append(line) ## Append line to the current section if it's set
  for key, value in sections.items():
    sections[key] = "\n".join(value) ## Join the lines back together
  return sections

chapter_16_sections = split_chapter(chapter_16_trimmed)

## Question Answering Mechanism

In [69]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Preprocess the user query using spaCy
def preprocess_query(query):
    # Parse the query with spaCy
    doc = nlp(query)
    # Normalize the query: lowercase, lemmatize, and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Define the sections variable with Chapter 1
sections = chapter_16_sections

# Define the QA mapping based on key phrases and corresponding sections
qa_mapping = {
    "amendment constitution":"amendment constitution",
    "parliamentary initiative":"parliamentary initiative",
    "popular initiative":"popular initiative"
    # Add more mappings as needed
}

# Update the Q&A system to use preprocessed queries
def answer_question_nlp(query, sections, qa_mapping):
    # Preprocess the user query
    processed_query = preprocess_query(query)
    
    # Search for a key in qa_mapping that matches the preprocessed query
    for key in qa_mapping:
        if key in processed_query:
            # Return the relevant section in the text
            return sections[key]
            
    return "Sorry, I couldn't find an answer to your question."

# Example usage
user_query = "What is the amendment of the constitution?"
answer = answer_question_nlp(user_query, sections, qa_mapping)
print(answer)

# Example 2
user_query_2 = 'Explain popular initiative'
answer_2 = answer_question_nlp(user_query_2, sections, qa_mapping)
answer_2


255. (1) A proposed amendment to this Constitution shall be
enacted in accordance with Article 256 or 257, and approved in
accordance with clause (2) by a referendum, if the amendment relates
to any of the following matters—
(a) the supremacy of this Constitution;
(b) the territory of Kenya;
(c) the sovereignty of the people;
(d) the national values and principles of governance referred to in
Article 10 (2) (a) to (d);
(e) the Bill of Rights;
(f) the term of office of the President;
(g) the independence of the Judiciary and the commissions and
independent offices to which Chapter Fifteen applies;
(h) the functions of Parliament;
Constitution of Kenya, 2010 155
(i) the objects, principles and structure of devolved government;
or
(j) the provisions of this Chapter.
(2) A proposed amendment shall be approved by a referendum
under clause (1) if—
(a) at least twenty per cent of the registered voters in each of at
least half of the counties vote in the referendum; and
(b) the amendment is su

''

# CHAPTER 17

In [70]:
chapter_17 = extract_specific_pages(pdf_path, 156,164 )
if "CHAPTER SEVENTEEN" in chapter_17:
    if "CHAPTER EIGHTEEN" in chapter_17:
        chapter_17_trimmed = chapter_17.split("CHAPTER SEVENTEEN", 1)[1].split("CHAPTER EIGHTEEN", 1)[0].strip()
    else:
        chapter_17_trimmed = chapter_17.split("CHAPTER SEVENTEEN", 1)[1].strip()


In [71]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Preprocess the user query using spaCy
def preprocess_query(query):
    # Parse the query with spaCy
    doc = nlp(query)
    # Normalize the query: lowercase, lemmatize, and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Example usage of the preprocessing function
user_query = "What is the supremacy of the constitution?"
processed_query = preprocess_query(user_query)
print(processed_query)

supremacy constitution


In [72]:
def split_chapter(chapter_text):
    # Split at key headings and strip extra whitespace
    sections = {
        "enforcement constitution": [],
        "construe constitution": [],
        "interpretation": []
    }

    # Split by new lines to process line by line
    lines = chapter_text.splitlines()

    current_section = None

    for line in lines:
        stripped_line = line.strip()

        if stripped_line.startswith("Enforcement of this Constitution"):
            current_section = "enforcement constitution"
        elif stripped_line.startswith("Construing this Constitution"):
            current_section = "construe constitution"
        elif stripped_line.startswith("Interpretation"):
            current_section = "interpretation"

        # Append line to the current section if it's set
        if current_section:
            sections[current_section].append(stripped_line)

    # Join each section into a single string
    for key in sections:
        sections[key] = "\n".join(sections[key])

    return sections

# Split the chapter into sections
chapter_17_sections = split_chapter(chapter_17_trimmed)

## Question Answer mechanism

In [73]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Preprocess the user query using spaCy
def preprocess_query(query):
    # Parse the query with spaCy
    doc = nlp(query)
    # Normalize the query: lowercase, lemmatize, and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Define the sections variable with Chapter 1
sections = chapter_17_sections

# Define the QA mapping based on key phrases and corresponding sections
qa_mapping = {
    "enforcement constitution": "enforcement constitution",
    "construe constitution": "construing constitution",
    "interpretation": "interpretation"
    
}

# Update the Q&A system to use preprocessed queries
def answer_question_nlp(query, sections, qa_mapping):
    # Preprocess the user query
    processed_query = preprocess_query(query)
    
    # Search for a key in qa_mapping that matches the preprocessed query
    for key in qa_mapping:
        if key in processed_query:
            # Return the relevant section in the text
            return sections[key]
            
    return "Sorry, I couldn't find an answer to your question."

# Example usage
user_query = "What is the enforcement of this Constitution?"
answer = answer_question_nlp(user_query, sections, qa_mapping)
print(answer)

# Example 2
user_query_2 = "explain what construing constitution means"
answer_2 = answer_question_nlp(user_query_2, sections, qa_mapping)
answer_2


Enforcement of this Constitution.
258. (1) Every person has the right to institute court proceedings,
claiming that this Constitution has been contravened, or is threatened
with contravention.
(2) In addition to a person acting in their own interest, court
proceedings under clause (1) may be instituted by—
(a) a person acting on behalf of another person who cannot act in
their own name;
(b) a person acting as a member of, or in the interest of, a group
or class of persons;
(c) a person acting in the public interest; or
(d) an association acting in the interest of one or more of its
members.


'Construing this Constitution.\n259. (1) This Constitution shall be interpreted in a manner that—\n(a) promotes its purposes, values and principles;\n(b) advances the rule of law, and the human rights and\nfundamental freedoms in the Bill of Rights;\n(c) permits the development of the law; and\n(d) contributes to good governance.\n(2) If there is a conflict between different language versions of\nthis Constitution, the English language version prevails.\n158 Constitution of Kenya, 2010\n(3) Every provision of this Constitution shall be construed\naccording to the doctrine of interpretation that the law is always\nspeaking and, therefore, among other things—\n(a) a function or power conferred by this Constitution on an office\nmay be performed or exercised as occasion requires, by the\nperson holding the office;\n(b) any reference in this Constitution to a State or other public\noffice or officer, or a person holding such an office, includes a\nreference to the person acting in or other

# CHAPTER 18

In [74]:
chapter_18 = extract_specific_pages(pdf_path, 163,165 )
chapter_18_cleaned = chapter_18.split("CHAPTER EIGHTEEN")[1].strip()
chapter_18_trimmed = chapter_18_cleaned.split('SCHEDULES')[0].strip()

In [75]:
def split_chapter(chapter_text):
    # Split at key headings and strip extra whitespace
    sections = {
        "consequential legislation": [],
        "consequential provision": [],
        "effective date": [],
        "repeal": []
    }

    # Split by new lines to process line by line
    lines = chapter_text.splitlines()

    current_section = None

    for line in lines:
        stripped_line = line.strip()

        # Check for headings and update current_section accordingly
        if stripped_line.startswith("Consequential legislation"):
            current_section = "consequential legislation"
        elif stripped_line.startswith("Transitional and consequential provisions"):
            current_section = "consequential provision"
        elif stripped_line.startswith("Effective Date"):  # Updated to match the heading case
            current_section = "effective date"
        elif stripped_line.startswith("Repeal of previous constitution"):
            current_section = "repeal"
        else:
            # Only append lines if there is an active section
            if current_section:
                sections[current_section].append(stripped_line)

    # Join each section into a single string, only if there are lines in the section
    for key in sections:
        sections[key] = "\n".join(sections[key]).strip()  # Use strip to remove leading/trailing whitespace

    return sections

# Split the chapter into sections
chapter_18_sections = split_chapter(chapter_18_trimmed)


# Question answering mechanism

In [76]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Preprocess the user query using spaCy
def preprocess_query(query):
    # Parse the query with spaCy
    doc = nlp(query)
    # Normalize the query: lowercase, lemmatize, and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Define the sections variable with Chapter 2
sections = chapter_18_sections

# Define the QA mapping based on key phrases and corresponding sections
# Define the qa mapping
qa_mapping = {
    "consequential legislation": "consequential legislation",
    "consequential provision": "consequential provision",
    "effective date": "effective date",
    "repeal": "repeal"
}



# Update the Q&A system to use preprocessed queries
def answer_question_nlp(query, sections, qa_mapping):
    # Preprocess the user query
    processed_query = preprocess_query(query)

    # Debug
    print(f"Processed query: {processed_query}")

    # Search for a key in qa_mapping that matches the preprocessed query
    for key in qa_mapping:
        if key in processed_query:
            # Return the relevant section in the text
            return sections[key]

    return "Sorry, I couldn't find an answer to your question."

# Example usage
user_query = "constitution repeal?"
answer = answer_question_nlp(user_query, sections, qa_mapping)
print(answer)
print('\n')
# example 2
user_query = "consequential provision?"
answer = answer_question_nlp(user_query, sections, qa_mapping)
print(answer)

Processed query: constitution repeal
264. Subject to the Sixth Schedule, for the avoidance of doubt,
the Constitution in force immediately before the effective date shall
stand repealed on the effective date.


Processed query: consequential provision
262. The transitional and consequential provisions set out in the
Sixth Schedule shall take effect on the effective date.


In [77]:
# Define the synonyms
synonyms = {
    "consequential legislation": ["consequential legislation", "related laws"],
    "consequential provision": ["consequential provision", "transitional provisions"],
    "effective date": ["effective date", "commencement date"],
    "repeal" : ["repeal", "constitution repeal", "repeal of the constitution"]
}

# Define the qa mapping
qa_mapping = {
    "consequential legislation": "consequential legislation",
    "consequential provision": "consequential provision",
    "effective date": "effective date",
    "repeal": "repeal"
}

sections = chapter_18_sections

# Preprocess the query
def preprocess_query(query):
    doc = nlp(query)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Function to correct the spelling
def correct_spelling(processed_query):
    spell = SpellChecker()
    words = processed_query.split()
    # Find misspelled words
    misspelled_words = spell.unknown(words)


    corrected_words = []
    for word in words:
        # Correct the word if it's misspelled
        if word in misspelled_words:
            corrected_word = spell.correction(word) # Get the most likely correction
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)

    # Reconstruct the initial sentence
    corrected_input = " ".join(corrected_words)

    return corrected_input

# Match with synonym support
def match_with_synonyms(query, qa_mapping, synonyms, sections):
    processed_query = preprocess_query(query)
    print(f"Processed Query: {processed_query}")  # Debugging line

    # Correct spelling in the processed query
    corrected_query = correct_spelling(processed_query)
    print(f"Corrected Query: {corrected_query}")  # Debugging line

    for key in qa_mapping.keys():
        print(f"Checking key: {key}")  # Debugging line

        # Get synonyms for current key
        for synonym in synonyms.get(key, [key]):
            print(f"Trying synonym: {synonym}")  # Debugging line
            if synonym in corrected_query:
                print(f"Match found with synonym: {synonym}")  # Debugging line
                # Return the section content corresponding to the found key
                section_content = sections.get(key, "Section not found.")
                print(f"Returning section content: {section_content}")  # Debugging line
                return section_content

    print("No match found")  # Debugging line if no match is found
    return None

# Answer function with synonym and fuzzy matching
def answer_question_nlp(query, sections, qa_mapping, synonyms):
    section_content = match_with_synonyms(query, qa_mapping, synonyms, sections)

    if section_content:
        return section_content  # Return the relevant section content

    return "Sorry, I couldn't find an answer to your question."

# Sample user query
user_query = "What is a repeal?"
answer = answer_question_nlp(user_query, sections, qa_mapping, synonyms)
print(answer)


Processed Query: repeal
Corrected Query: repeal
Checking key: consequential legislation
Trying synonym: consequential legislation
Trying synonym: related laws
Checking key: consequential provision
Trying synonym: consequential provision
Trying synonym: transitional provisions
Checking key: effective date
Trying synonym: effective date
Trying synonym: commencement date
Checking key: repeal
Trying synonym: repeal
Match found with synonym: repeal
Returning section content: 264. Subject to the Sixth Schedule, for the avoidance of doubt,
the Constitution in force immediately before the effective date shall
stand repealed on the effective date.
264. Subject to the Sixth Schedule, for the avoidance of doubt,
the Constitution in force immediately before the effective date shall
stand repealed on the effective date.


# Cumulative sections, qa_mapping and synonyms

In [78]:
# Define the synonyms explicitly so that we can easily reference them later
synonyms = {
    "supremacy": ["supremacy", "authority", "ultimate power"],
    "sovereignty": ["sovereignty", "power of the people", "authority of the people", "self rule", "autonomy"],
    "defence": ["defense", "protection", "preservation"],
    "declaration": ["declaration", "proclamation", "statement", "announcement", "affirmation"],
    "territory": ["territory", "land", "region", "area", "jurisdiction", "bounds"],
    "devolution": ["devolution", "decentralization", "delegation", "transfer of power", "local governance", "subsidiarity"],
    "languages": ["languages", "tongues", "dialects", "official languages", "linguistic diversity"],
    "religion": ["religion", "faith", "belief systems", "spiritual practice", "secularism", "church-state separation"],
    "symbol": ["symbol", "emblem", "insignia", "representation", "national icon"],
    "day": ["day", "holiday", "observance", "public holiday", "commemoration", "remembrance"],
    "value": ["value", "principle", "ethic", "core value", "standard", "national ideal"],
    "governance": ["governance", "government", "administration", "management", "public service", "political structure"],
    "culture": ["culture", "heritage", "tradition", "customs", "societal norms", "arts"],
    "entitlement": ["entitlement", "right", "eligibility", "entitlement rights", "benefits", "privileges"],
    "retention": ["retention", "maintenance", "keeping", "preservation", "continuation"],
    "birth": ["birth", "nativity", "origin", "ancestry", "inborn citizenship"],
    "registration": ["registration", "enlistment", "enrollment", "citizenship application", "naturalization"],
    "dual": ["dual", "multiple", "dual nationality", "two-fold citizenship"],
    "revocation": ["revocation", "cancellation", "annulment", "rescission", "forfeiture", "withdrawal"],
    "legislation": ["legislation", "laws", "legal framework", "statutes", "enactment"],
    "fundamental": ["fundamental", "fundamental freedoms", "essential"],
    "application": ["application", "exercise", "freedom enforcement"],
    "implementation": ["implementation", "right support", "execution"],
    "authority": ["authority", "judicial power", "court jurisdiction", "judicial review"],
    "limitation": ["limitation", "freedom restriction", "limit entitlement"],
    "limit": ["limit","absolute right", "inalienable freedom", "immutable right"],
    "life": ["life", "existence", "entitlement to life", "survive"],
    "equality": ["equality", "equal treatment", "justice"],
    "dignity": ["dignity", "intrinsic respect", "inherent value", "personal honor"],
    "security": ["security", "liberty", "personal safety"],
    "slavery": ["slavery", "bondage", "force servitude", "involuntary labor", "coerce work"],
    "privacy": ["privacy", "confidential", "personal space"],
    "conscience": ["conscience", "liberty of thought", "religion freedom", "belief"],
    "expression": ["expression", "communication", "speak"],
    "medium": ["medium", "press autonomy", "journalist freedom", "right to information"],
    "information": ["information", "transparent", "datum"],
    "association": ["association", "union", "group formation", "association right"],
    "assembly": ["assembly", "dissent", "peace assembly", "demonstrate", "petition right"],
    "political": ["political", "electoral right", "political engagement", "vote right"],
    "movement": ["movement", "migration", "resident freedom", "movement liberty"],
    "Property": ["Property", "property safeguard", "assets security"],
    "work": ["work", "benefit", "employ"],
    "environment": ["environment", "conserve", "right to a healthy environment"],
    "economic": ["economic", "standard of life", "social welfare"],
    "language": ["language", "cultural right", "linguistic freedom", "cultural expression"],
    "family": ["family", "society foundation", "parent right"],
    "consumer": ["consumer", "client entitlement", "buyer safeguard"],
    "administrative": ["administrative", "fair administration rule", "legal administration measure"],
    "justice": ["justice", "legal access", "justice available"],
    "arrest": ["arrest", "capture", "seizure", "apprehension"],
    "fair hearing": ["fair hearing", "prosecute", "equity hearing"],
    "custody": ["custody", "confinement", "imprisonment", "prison"],
    "interpret": ["interpret", "explain", "clarify", "overview"],
    "infant": ["infant", "child", "kid", "toddler"],
    "disable": ["disable", "handicap", "impairment", "challenge"],
    "youth": ["youth", "young adult", "adolescent"],
    "minority": ["minority", "diversity right", "affirmative right"],
    "old": ["old", "elder", "vintage"],
    "emergency": ["emergency", "danger", "crisis", "disaster"],
    "national": ["national", "human right", "equaity Body", "right authority"],
    "principle": ["principle", "land management principle", "land policy guide"],
    "classification": ["classification", "land category", "types of land"],
    "public": ["public", "government land", "state property", "national land", "public landhold"],
    "community": ["community", "ethnic land", "cultural landhold", "community land"],
    "private": ["private", "individual landhold", "personal property", "freehold land", "private land"],
    "landhold" : ["landhold", "foreign lease", "alien land", "non-citizen land"],
    "regulation" : ["regulation", "land use policy", "property regulation", "land oversight"],
    "commission" : ["commission", "land authority", "public land commission", "land policy agency"],
    "land" : ["land", "land legislation", "property law", "land-use regulation", "land act"],
    "obligation" : ["obligation", "biodiversity duty", "conservation mandate"],
    "environmental" : ["environmental", "ecological redress", "sustainability enforcement"],
    "resource" : ["resource", "resource use agreement", "environmental concession"],
    "legislation" : ["legislation", "environmental law", "green policy enactment"],
    "responsibility": ["responsibility", "accountable", "commitment", "charge"],
    "oath": ["oath", "swear", "state office affirmation", "vow", "declare"],
    "conduct": ["conduct", "behavior", "comportment", "demeanor"],
    "finance": ["finance", "finance integrity", "wealth", "invest", "finance management"],
    "restrict": ["restrict", "constrain", "regulate", "prohibit", "curtail", "control"],
    "citizen": ["citizen", "inhabitant", "local resident"],
    "establish": ["establish", "corrupt", "bribe", "fraud"],
    "leader": ["leader", "governing, authority"],
    "judicial authority": ["judicial authority","legal authority","court jurisdiction"],
    "judicial independence": ["judicial independence","judicial autonomy","judicial impartiality"],
    "judicial offices": ["judicial office","legal position","judicial role"],
    "court systems": ["court system","judicial system","legal system"],
    "supreme court": ["supreme court","highest court","apex court"],
    "appeal court": ["appeal court","appellate court","review court"],
    "high court": ["high court","superior court"],
    "judicial appointments": ["judicial appointment","judge appointment","judicial selection"],
    "office tenure": ["office tenure","office term","service duration"],
    "office removal": ["office removal","office dismissal","office termination"],
    "subordinate courts": ["subordinate court","local court","inferior court"],
    "jsc establishment": ["judicial service commission establishment","judicial service commission creation","judicial service commission formation"],
    "jsc functions": ["judicial service commission function","judicial service commission role","judicial service commission responsibility"],
    "judiciary fund": ["judiciary fund","judicial fund","legal fund"],
    "object devolution": ["object devolution", "power decentralization", "government devolution"],
    "devolve government": ["devolve government", "decentralization principle"],
    "county government": ["county government", "local government", "regional administration"],
    "membership": ["membership", "composition", "structure"],
    "speaker": ["speaker", "assembly speaker", "preside officer"],
    "county executive committee": ["county executive committee", "regional executive body", "local executive board"],
    "county election": ["county election", "local election", "regional election"],
    "county removal": ["county removal", "county office dismissal", "regional removal"],
    "vacancy": ["vacancy", "position vacancy", "office vacancy"],
    "functions of county executive committees": ["functions of county executive committees", "duties of county executive commitees", "county executive commitees roles"],
    "city": ["city", "urban center", "municipality"],
    "legislative authority": ["legislative authority", "law make power", "regulatory authority"],
    "function and power of national and county government": ["function and power of national and county government", "government duty", "government responsibility"],
    "transfer": ["transfer", "delegation", "reassignment"],
    "county boundary": ["county boundary", "county border", "regional limit"],
    "cooperation": ["cooperation", "collaboration", "coordination"],
    "support": ["support", "assistance", "aid"],
    "conflict of law": ["conflict of law", "legal discrepancy", "jurisdiction conflict"],
    "suspension": ["suspension", "interruption", "temporary halt"],
    "qualification": ["qualification", "eligibility", "requirement"],
    "vacation": ["vacation", "office leave", "office absence"],
    "summon witness": ["summon witness", "convene witness", "request witness attendance"],
    "public participation county assembly power privilege immunity": ["public participation county assembly power privilege immunity", "county assembly right", "local governance privilege"],
    "gender balance": ["gender balance", "gender equity", "equal representation"],
    "county transition": ["county transition", "regional handover", "local government transition"],
    "publication": ["publication", "official release", "public announcement"],
    "legislation": ["legislation", "regulation", "law"],
    "principle public finance": ["principle public finance", "public finance guideline", "fundamental financial rule"],
    "equitable sharing of national revenue": ["equitable share of national revenue", "fair distribution of revenue"],
    "equitable share law": ["equitable share law", "financial law", "equitable distribution law"],
    "equalisation fund": ["equalisation fund", "equalization", "equal opportunity fund"],
    "consultation": ["consultation", "discussion", "advisement"],
    "consolidated fund": ["consolidated fund", "main fund", "central fund"],
    "revenue funds": ["revenue funds", "revenue", "income fund"],
    "contingencies fund": ["contingency fund", "reserve fund", "emergency fund"],
    "tax impose": ["tax impose", "tax authority", "tax power"],
    "national government borrow": ["national government borrow", "national debt", "federal borrow"],
    "loan guarantee": ["loan guarantee", "credit guarantee", "loan assurance"],
    "public debt": ["public debt", "national debt", "government debt"],
    "commission on revenue allocation": ["commission on revenue allocation", "allocation commission", "revenue allocation committee"],
    "division of revenue": ["division of revenue", "revenue distribution", "allocation of revenue"],
    "budget content": ["budget content", "budget structure", "budget framework"],
    "expenditure": ["expenditure", "cost", "expense"],
    "supplementary appropriation": ["supplementary appropriation", "additional appropriation", "extra allocation"],
    "financial control": ["financial control", "financial oversight", "budget control"],
    "procurement": ["procurement", "acquisition", "purchase"],
    "budget contoller": ["budget contoller", "cost controller"],
    "auditor-general": ["auditor general", "chief auditor", "head auditor"],
    "salary": ["salary","remuneration", "pay"],
    "central bank of kenya": ["central bank of kenya", "central bank", "central financial authority"],
    "principle": ["principle", "values and principles of public service", "public service values", "core principles of public service"],
    "public service commission": ["public service commission", "the public service commission", "public service authority", "government service commission"],
    "functions": ["function", "functions powers public service commission", "role the public service commission", "duties public service commission"],
    "staffing": ["staffing", "staffing of county governments", "county government staffing", "human resources for county governments"],
    "protection": ["protection", "protection of public officers", "safeguarding public officers", "security for public officials"],
    "teachers service commission": ["teachers service commission", "commission for teachers", "educators service commission"],
    "national security": ["national security", "principles of national security"],
    "national security organs": ["national security organs", "security organs", "national security agencies"],
    "national security council": ["national security council", "establishment of the national security council", "national security council formation", "creation of the national security council"],
    "defence forces": ["defence forces and defence council", "defence forces establishment", "formation of defence forces and council"],
    "national intelligence service": ["national intelligence service", "establishment of national intelligence service", "national intelligence service creation", "formation of national intelligence service"],
    "national police service": ["national police service", "establishment of the national police service", "national police service formation", "creation of the national police service"],
    "functions national police service": ["functions national police service", "objects and functions of the national police service", "roles of the national police service", "duties and functions of the national police"],
    "command national police service": ["command national police service", "command of the national police service", "national police service command", "leadership of the national police service"],
    "national police service commission": ["national police service commission", "police service commission", "commission for the national police service"],
    "police service": ["police service", "additional police service", "alternative police service"],
    "application of chapter": ["application of chapter", "chapter application", "provisions of the chapter"],
    "commission independent office": ["commission independent office", "commissions and independent offices", "independent commissions", "commission offices"],
    "term office": ["term office", "term office", "office term", "tenure condition"],
    "removal office": ["removal from office", "dismissal office", "termination office"],
    "general function power": ["general function power", "overall power", "general power functions"],
    "incorporation commission independent office": ["incorporation commission independent office", "incorporation commissionindependent office", "establishment commission", "set up independent office"],
    "reporting commission independent office": ["reporting commission independent office", "reporting commission independent office", "commissions report", "report independent offices"]


}

# Define qa mapping explicitly too
qa_mapping = {
    "supremacy": "supremacy",
    "sovereignty": "sovereignty",
    "defence": "defence",
    "declaration": "declaration",
    "territory": "territory", 
    "devolution": "devolution",
    "language": "languages",
    "religion": "religion",
    "symbol": "symbol",
    "day": "day",
    "value": "value",
    "culture": "culture",
    "entitlement": "entitlement",
    "retention": "retention",  
    "birth": "birth",
    "registration": "registration",
    "dual": "dual",
    "revocation": "revocation",
    "legislation": "legislation",
    "fundamental": "fundamental",
    "application": "application",
    "implementation": "implementation",
    "authority": "authority",
    "limitation": "limitation",
    "limit": "limit",
    "life": "life",
    "equality": "equality",
    "dignity": "dignity",
    "security": "security",
    "slavery": "slavery",
    "privacy": "privacy",
    "conscience": "conscience",
    "expression": "expression",
    "medium": "medium",
    "information": "information",
    "association": "association",
    "assembly": "assembly",
    "political": "political",
    "movement": "movement",
    "property": "property",
    "work": "work",
    "environment": "environment",
    "economic": "economic",
    "language": "language",
    "family": "family",
    "consumer": "consumer",
    "administrative": "administrative",
    "justice": "justice",
    "arrest": "arrest",
    "fair hearing": "fair hearing",
    "custody": "custody",
    "interpret": "interpret",
    "infant": "infant",
    "disable": "disable",
    "youth": "youth",
    "minority": "minority",
    "old": "old",
    "emergency": "emergency",
    "national": "national",
    "principle": "principle",
    "classification": "classification",
    "public": "public",
    "community": "community",
    "private": "private",
    "landhold": "landhold",
    "regulation": "regulation",
    "commission": "commission",
    "land": "land",
    "obligation": "obligation",
    "environmental": "environmental",
    "resource": "resource",
    "legislation": "legislation",
    "responsibility": "responsibility",
    "oath": "oath",
    "conduct": "conduct",
    "finance": "finance",
    "restrict": "restrict",
    "citizen": "citizen",
    "establish": "establish",
    "leader": "leader",
    "judicial authority": "judicial authority",
    "judicial independence" :"judiciary independence",
    "judicial offices": "judicial offices",
    "court systems": "court systems",
    "supreme court": "supreme court",
    "appeal court": "appeal court",
    "high court": "high court",
    "judicial appointments": "judicial appointments",
    "office tenure": "office tenure",
    "office removal": "office removal",
    "subordinate courts": "subordinate courts",
    "jsc establishment": "jsc establishment",
    "jsc functions": "jsc functions",
    "judiciary fund": "judiciary fund",
    "object devolution": "object devolution",
    "devolve government": "devolve government",
    "county government": "county government",
    "membership": "membership",
    "speaker": "speaker",
    "county executive committee": "county executive committee",
    "county election": "county election",
    "county removal": "county removal",
    "vacancy": "vacancy",
    "functions of county executive committees": "executive function",
    "city": "city",
    "legislative authority": "legislative authority",
    "function and power of national and county government": "function and power of national and county government",
    "transfer": "transfer",
    "county boundary": "county boundary",
    "cooperation": "cooperation",
    "support": "support",
    "conflict of law": "law conflict",
    "suspension": "suspension county government",
    "qualification": "qualification",
    "vacation": "vacation",
    "summon witness": "summon witness",
    "public participation county assembly power privilege immunity": "public participation county assembly power privilege immunity",
    "gender balance": "gender balance",
    "county transition": "transition",
    "publication": "publication",
    "legislation": "legislation",
    "principle public finance": "principle public finance",
    "equitable sharing of national revenue": "equitable sharing of national revenue",
    "equitable share law": "equitable share",
    "equalisation fund": "equalisation fund",
    "consultation": "consultation",
    "consolidated fund": "consolidate",
    "revenue funds": "revenue",
    "contingencies fund": "contingency",
    "tax impose": "tax",
    "national government borrow": "borrow",
    "loan guarantee": "loan",
    "public debt": "debt",
    "commission on revenue allocation": "commission",
    "division of revenue": "division",
    "budget content": "budget",
    "expenditure": "expenditure",
    "supplementary appropriation": "supplementary",
    "financial control": "control",
    "procurement": "procurement",
    "budget controller": "budget controller",
    "auditor-general": "auditor",
    "salary": "salary",
    "central bank of kenya": "bank",
    "principle": "principle",
    "public service commission": "public service commission",
    "functions": "functions",
    "staffing": "staffing",
    "protection": "protection",
    "teachers service commission": "teachers service commission",
    "national security": "national security",
    "national security organs": "national security organs",
    "national security council": "national security council",
    "defence forces": "defence forces",
    "national intelligence service": "national intelligence service",
    "national police service": "national police service",
    "functions national police service": "functions national police service",
    "command national police service": "command national police service",
    "national police service commission": "national police service commission",
    "police service": "police service",
    "application of chapter": "application of chapter",
    "commission independent office": "commission independent office",
    "term of office": "term of office",
    "removal from office": "removal from office",
    "general function power": "general function power",
    "incorporation commission independent office": "incorporation commission independent office",
    "reporting commission independent office": "reporting commission independent office",
    "amendment constitution":"amendment constitution",
    "parliamentary initiative":"parliamentary initiative",
    "popular initiative":"popular initiative",
    "enforcement constitution": "enforcement constitution",
    "construe constitution": "construing constitution",
    "interpretation": "interpretation",
    "consequential legislation": "consequential legislation",
    "consequential provision": "consequential provision",
    "effective date": "effective date",
    "repeal": "repeal"

}

combined_sections = {**chapter_1_sections, **chapter_2_sections, **chapter_3_sections, **chapter_4_sections,
                     **chapter_5_sections, **chapter_6_sections, **chapter_10_sections, **chapter_11_sections,
                     **chapter_12_sections, **chapter_13_sections, **chapter_14_sections, **chapter_15_sections,
                     **chapter_16_sections, **chapter_17_sections, **chapter_18_sections}
# Print the keys of the dictionary
len(combined_sections.keys())

183

# The function below is now the main code and can be integrated with the telegram chatbot interface

In [79]:
# Load the language model (make sure to have spaCy installed and the model downloaded)
nlp = spacy.load("en_core_web_sm")

# Define synonym mapping
synonyms = synonyms

sections = combined_sections

# QA mapping
qa_mapping = qa_mapping

# Preprocess the query
def preprocess_query(query):
    doc = nlp(query)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Function to correct the spelling
def correct_spelling(processed_query):
    spell = SpellChecker()
    words = processed_query.split()
    # Find misspelled words
    misspelled_words = spell.unknown(words)

    corrected_words = []
    for word in words:
        # Correct the word if it's misspelled
        if word in misspelled_words:
            corrected_word = spell.correction(word)  # Get the most likely correction
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)

    # Reconstruct the initial sentence
    corrected_input = " ".join(corrected_words)
    return corrected_input

# Match with synonym support
def match_with_synonyms(query, qa_mapping, synonyms, citizenship_mapping):
    processed_query = preprocess_query(query)
    print(f"Processed Query: {processed_query}")  # Debugging line

    # Correct spelling in the processed query
    corrected_query = correct_spelling(processed_query)
    print(f"Corrected Query: {corrected_query}")  # Debugging line

    # Check for specific citizenship subtopics first
    for subtopic, section_key in citizenship_mapping.items():
        if subtopic in corrected_query:
            print(f"Specific citizenship topic detected: {subtopic}")
            return section_key  # Return the section key for the specific subtopic

    # Only check for general citizenship if no specific subtopic was found
    if "citizenship" in corrected_query:
        print("General citizenship query detected.")  # Debugging line
        return "citizenship"  # Return a placeholder indicating interest in citizenship

    for key in qa_mapping:
        print(f"Checking key: {key}")  # Debugging line
        for synonym in synonyms_3.get(key, [key]):
            print(f"Trying synonym: {synonym}")  # Debugging line
            if synonym in corrected_query:
                print(f"Match found with synonym: {synonym}")  # Debugging line
                return key  # Only return section value of the key

    print("No match found")  # Debugging line if no match is found
    return None

# Answer function with synonym and fuzzy matching
def answer_question_nlp(query, sections, qa_mapping, synonyms):
    section_key = match_with_synonyms(query, qa_mapping, synonyms, citizenship_mapping)
    
    if section_key in sections:
        # Retrieve the relevant section from the specified chapter
        return sections.get(section_key, "Section not found.")
    elif section_key == "citizenship":
        return (f"It seems you're interested in citizenship. "
                f"Available subtopics include: {list((citizenship_mapping.keys()))}.")
    
    return "Sorry, I couldn't find an answer to your question."

# Example
user_query = "What about police services?"
answer = answer_question_nlp(user_query, sections, qa_mapping, synonyms)
print(answer)


Processed Query: police service
Corrected Query: police service
Checking key: supremacy
Trying synonym: supremacy
Checking key: sovereignty
Trying synonym: sovereignty
Checking key: defence
Trying synonym: defence
Checking key: declaration
Trying synonym: declaration
Checking key: territory
Trying synonym: territory
Checking key: devolution
Trying synonym: devolution
Checking key: language
Trying synonym: language
Checking key: religion
Trying synonym: religion
Checking key: symbol
Trying synonym: symbol
Checking key: day
Trying synonym: day
Checking key: value
Trying synonym: value
Checking key: culture
Trying synonym: culture
Checking key: entitlement
Trying synonym: entitlement
Checking key: retention
Trying synonym: retention
Checking key: birth
Trying synonym: birth
Checking key: registration
Trying synonym: registration
Checking key: dual
Trying synonym: dual
Checking key: revocation
Trying synonym: revocation
Checking key: legislation
Trying synonym: legislation
Checking key: fu

# Deployment test

In [ ]:
import spacy
from spellchecker import SpellChecker
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, CallbackContext
import nest_asyncio
import asyncio
import os
from dotenv import load_dotenv
load_dotenv()  # Loads the .env file
bot_token = os.getenv("BOT_TOKEN")

# Apply the nest_asyncio patch
nest_asyncio.apply()

# Initialize spaCy and load the language model
nlp = spacy.load("en_core_web_sm")

# Initialize spell checker
spell = SpellChecker()

# Function to preprocess query
def preprocess_query(query):
    doc = nlp(query)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Function to correct spelling
def correct_spelling(processed_query):
    words = processed_query.split()
    misspelled_words = spell.unknown(words)
    corrected_words = [spell.correction(word) if word in misspelled_words else word for word in words]
    return " ".join(corrected_words)

# Function to match with synonym support
def match_with_synonyms(query, qa_mapping, synonyms, citizenship_mapping):
    processed_query = preprocess_query(query)
    corrected_query = correct_spelling(processed_query)

    # Check for specific citizenship subtopics first
    for subtopic, section_key in citizenship_mapping.items():
        if subtopic in corrected_query:
            return section_key

    # General citizenship check
    if "citizenship" in corrected_query:
        return "citizenship"

    # Synonym matching
    for key in qa_mapping:
        for synonym in synonyms.get(key, [key]):
            if synonym in corrected_query:
                return key
    return None

# Answer function
def answer_question_nlp(query):
    section_key = match_with_synonyms(query, qa_mapping, synonyms, citizenship_mapping)
    if section_key in sections:
        return sections.get(section_key, "Section not found.")
    elif section_key == "citizenship":
        return (f"It seems you're interested in citizenship. "
                f"Available subtopics include: {list(citizenship_mapping.keys())}.")
    return "Sorry, I couldn't find an answer to your question."

# Handler for messages
async def handle_message(update: Update, context: CallbackContext) -> None:
    user_query = update.message.text
    answer = answer_question_nlp(user_query)
    await update.message.reply_text(answer)

# Main function to set up the bot
async def main() -> None:
    # Replace 'YOUR_BOT_TOKEN' with your actual bot token
    application = ApplicationBuilder().token(bot_token).build()
    
    # Adding the message handler for text messages
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
    
    # Start the bot
    await application.initialize()
    await application.start()
    await application.updater.start_polling()

# Run the bot in Jupyter
asyncio.run(main())


Exception happened while polling for updates.
Traceback (most recent call last):
  File "e:\SCHOOL\anaconda3.1\envs\Mubea-env\lib\site-packages\telegram\ext\_updater.py", line 753, in _network_loop_retry
    if not await do_action():
  File "e:\SCHOOL\anaconda3.1\envs\Mubea-env\lib\site-packages\telegram\ext\_updater.py", line 747, in do_action
    return action_cb_task.result()
  File "e:\SCHOOL\anaconda3.1\envs\Mubea-env\lib\asyncio\futures.py", line 178, in result
    raise self._exception
  File "e:\SCHOOL\anaconda3.1\envs\Mubea-env\lib\asyncio\tasks.py", line 280, in __step
    result = coro.send(None)
  File "e:\SCHOOL\anaconda3.1\envs\Mubea-env\lib\site-packages\telegram\ext\_updater.py", line 377, in polling_action_cb
    updates = await self.bot.get_updates(
  File "e:\SCHOOL\anaconda3.1\envs\Mubea-env\lib\site-packages\telegram\ext\_extbot.py", line 649, in get_updates
    updates = await super().get_updates(
  File "e:\SCHOOL\anaconda3.1\envs\Mubea-env\lib\site-packages\tele

# I've tested the deployment on telegram and it works prefectly. We'll begin tuning the bot on Monday, 4th November.